# Analisi Finanziaria e Simulazione Monte Carlo ETF

## 1. Installazione Librerie

In [ ]:
!pip install gspread arch python-dotenv openpyxl ipywidgets statsmodels --quiet
%matplotlib inline
!pip show plotly
!pip install --upgrade plotly



Name: plotly
Version: 6.1.2
Summary: An open-source interactive data visualization library for Python
Home-page: https://plotly.com/python/
Author: 
Author-email: Chris P <chris@plot.ly>
License: MIT License

Copyright (c) 2016-2024 Plotly Technologies Inc.

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in
all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOS

## 2. Importazioni Principali

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import datetime
from datetime import date # Explicit import for date.today()
import os
import io
import warnings

from arch import arch_model
from arch.__future__ import reindexing # For Arch library
from scipy.stats import t, chi2, gennorm
from statsmodels.stats.diagnostic import acorr_ljungbox
# Aggiungi questo import insieme agli altri
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

import ipywidgets as widgets
from IPython.display import display, clear_output, Javascript, HTML
import plotly.graph_objects as go
import plotly.io as pio

# ♦ Colab
try:
    pio.renderers.default = "colab"
# ♦ Jupyter-Lab / classico
except ValueError:
    pio.renderers.default = "notebook"                           #"notebook_connected"


from google.colab import drive
import gspread
from google.oauth2.service_account import Credentials
from dotenv import load_dotenv

warnings.filterwarnings("ignore")


In [ ]:

# CELLA 7 (ID: e9ba5be3) - Funzione plot_prezzi_simulati (CORRETTA)
import plotly.graph_objects as go
def plot_prezzi_simulati_plotly(simulated_prices, titolo="Andamento Prezzi Simulati", start_date=None, historical_prices=None):
    """Genera un fan chart interattivo dei prezzi simulati usando Plotly."""
    if simulated_prices is None or simulated_prices.size == 0:
        print("Nessun prezzo simulato da plottare.")
        return None

    if start_date is None:
        start_date = pd.Timestamp.today()

    giorni_forecast = simulated_prices.shape[0]
    forecast_dates_axis = pd.bdate_range(start=start_date, periods=giorni_forecast, freq='B')

    quantili_plot = [5, 25, 50, 75, 95]
    price_percentiles = np.percentile(simulated_prices, quantili_plot, axis=1)

    fig = go.Figure()

    p_map = {q: price_percentiles[i] for i, q in enumerate(quantili_plot)}

    # Fascia esterna (5-95)
    fig.add_trace(go.Scatter(
        x=np.concatenate([forecast_dates_axis, forecast_dates_axis[::-1]]),
        y=np.concatenate([p_map[95], p_map[5][::-1]]),
        fill='toself',
        fillcolor='rgba(0,100,200,0.2)',
        line=dict(width=0),
        name='5-95 Percentile',
        hoverinfo='skip'
    ))
    # Fascia interna (25-75, IQR)
    fig.add_trace(go.Scatter(
        x=np.concatenate([forecast_dates_axis, forecast_dates_axis[::-1]]),
        y=np.concatenate([p_map[75], p_map[25][::-1]]),
        fill='toself',
        fillcolor='rgba(0,100,200,0.4)',
        line=dict(width=0),
        name='25-75 Percentile (IQR)',
        hoverinfo='skip'
    ))

    # Linea della Mediana (50° percentile)
    fig.add_trace(go.Scatter(
        x=forecast_dates_axis,
        y=p_map[50],
        mode='lines',
        name='Mediana (50%)',
        line=dict(color='navy', width=2)
    ))

    # Prezzi storici
    if historical_prices is not None and not historical_prices.empty:
        fig.add_trace(go.Scatter(
            x=historical_prices.index,
            y=historical_prices,
            mode='lines',
            name='Prezzi Storici',
            line=dict(color='black', width=2)
        ))

    fig.update_layout(
        title_text=titolo,
        xaxis_title="Data",
        yaxis_title="Prezzo Stimato",
        template="plotly_white",
        height=500,
        showlegend=True,
        hovermode="x unified"
    )

    return fig # Ritorna l'oggetto figura Plotly



## 3. Setup Ambiente Google Colab, API Keys e Google Sheets

In [ ]:
# Mount Google Drive
print("Montaggio di Google Drive...")
try:
    drive.mount('/content/drive', force_remount=True)
    print("Google Drive montato con successo.")
except Exception as e:
    print(f"Errore durante il montaggio: {e}")

# Percorso progetto su Google Drive (modifica se serve)
project_path = '/content/drive/My Drive/Colab Notebooks' # Assicurati che questa cartella esista
os.makedirs(project_path, exist_ok=True)
# os.chdir(project_path) # Changing directory can sometimes cause issues with relative paths for service account json
print(f"Percorso di progetto impostato su: {project_path}")
print(f"Directory di lavoro corrente: {os.getcwd()}")

# Caricamento API da file .env
print("\nCaricamento delle chiavi API...")
env_file_path = os.path.join(project_path, '.env')
if os.path.exists(env_file_path):
    try:
        load_dotenv(env_file_path)
        print("File .env caricato.")
    except Exception as e:
        print(f"Errore nel caricamento del file .env: {e}")
else:
    print(f"File .env non trovato in {env_file_path}. Assicurati che esista o inserisci le API keys manualmente.")

# Estrazione chiavi API
FMP_API_KEY = os.getenv('FMP_API_KEY')
ALPHA_VANTAGE_API_KEY = os.getenv('ALPHA_VANTAGE_API_KEY')
TWELVE_DATA_API_KEY = os.getenv('TWELVE_DATA_API_KEY')
TIINGO_API_KEY = os.getenv('TIINGO_API_KEY')
EOD_API_KEY = os.getenv('EOD_API_KEY')

print(f"FMP_API_KEY: {'✔️' if FMP_API_KEY else '❌'}")
print(f"ALPHA_VANTAGE_API_KEY: {'✔️' if ALPHA_VANTAGE_API_KEY else '❌'}")
print(f"TWELVE_DATA_API_KEY: {'✔️' if TWELVE_DATA_API_KEY else '❌'}")
print(f"TIINGO_API_KEY: {'✔️' if TIINGO_API_KEY else '❌'}")
print(f"EOD_API_KEY: {'✔️' if EOD_API_KEY else '❌'}")

# --- Config Google Sheets ---
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
# IMPORTANTE: Assicurati che il file JSON sia nel percorso corretto o fornisci il percorso completo.
SERVICE_ACCOUNT_FILE_NAME = 'ecstatic-acumen-459621-r1-994994eba32e.json' # Nome del file JSON
SERVICE_ACCOUNT_FILE = os.path.join(project_path, SERVICE_ACCOUNT_FILE_NAME) # Percorso completo al file JSON
SHEET_ID = '1ukhNTZC0eJpfFF4V1_S9ZN19jM5xqzjhHx2V3PD8leo' # IL TUO SPREADSHEET ID QUI
WORKSHEET_NAME = 'ETF_EIMI' # IL NOME ESATTO DEL FOGLIO NEL TUO SPREADSHEET QUI

print(f"\nVerifica file credenziali Google Sheets: {'✔️ Esiste' if os.path.exists(SERVICE_ACCOUNT_FILE) else '❌ NON TROVATO in ' + SERVICE_ACCOUNT_FILE}")


Montaggio di Google Drive...
Mounted at /content/drive
Google Drive montato con successo.
Percorso di progetto impostato su: /content/drive/My Drive/Colab Notebooks
Directory di lavoro corrente: /content

Caricamento delle chiavi API...
File .env caricato.
FMP_API_KEY: ✔️
ALPHA_VANTAGE_API_KEY: ✔️
TWELVE_DATA_API_KEY: ✔️
TIINGO_API_KEY: ✔️
EOD_API_KEY: ✔️

Verifica file credenziali Google Sheets: ✔️ Esiste


## 4. Funzioni di Caricamento Dati

In [ ]:

# Upload credenziali JSON per Google Sheets (se non monti Drive)
from ipywidgets import FileUpload
from IPython.display import display, HTML
import os

SCOPES = ["https://www.googleapis.com/auth/spreadsheets.readonly"]

upload_creds = FileUpload(accept=".json", multiple=False, description="Carica credenziali JSON")
display(HTML("<h4>📁 Carica il file di Service Account (JSON) per Google Sheets:</h4>"))
display(upload_creds)

def salva_credenziali(change):
    uploaded_filename = list(upload_creds.value.keys())[0]
    content = upload_creds.value[uploaded_filename]["content"]
    with open(uploaded_filename, "wb") as f:
        f.write(content)
    global SERVICE_ACCOUNT_FILE
    SERVICE_ACCOUNT_FILE = uploaded_filename
    print(f"✅ File credenziali salvato come: {uploaded_filename}")

upload_creds.observe(salva_credenziali, names="value")


FileUpload(value={}, accept='.json', description='Carica credenziali JSON')

In [ ]:

def _load_data_from_google_sheets(sheet_id, worksheet_name, service_account_file_path=None):
    """
    Carica i dati da Google Sheets, cercando di ottenere valori non formattati o stringhe
    per gestire correttamente i decimali con virgola.
    """
    print(f"Caricamento dati da Google Sheet ID: '{sheet_id}', foglio: '{worksheet_name}'...")
    creds_path = service_account_file_path or globals().get("SERVICE_ACCOUNT_FILE", None)
    if creds_path is None or not os.path.exists(creds_path):
        print("❌ Il file delle credenziali non è stato caricato o non esiste.")
        return None
    try:
        from google.oauth2 import service_account
        import gspread
        creds = service_account.Credentials.from_service_account_file(creds_path, scopes=SCOPES)
        gc = gspread.authorize(creds)
        spreadsheet = gc.open_by_key(sheet_id)
        worksheet = spreadsheet.worksheet(worksheet_name)

        # Legge tutti i valori, la prima riga sarà l'header
        # value_render_option='UNFORMATTED_VALUE' cerca di prendere il numero sottostante (es. 32.35)
        # value_render_option='FORMATTED_VALUE' prende la stringa come la vedi (es. "32,35")
        # Proviamo prima con UNFORMATTED_VALUE, se fallisce o dà ancora interi, FORMATTED_VALUE potrebbe essere un'alternativa
        # per ora usiamo il default di get_all_values() che spesso sono stringhe.

        list_of_lists = worksheet.get_all_values() # Restituisce una lista di liste (righe)

        # ---> AGGIUNGI QUESTO DEBUG <---
        print(f"--- DEBUG (load_google_sheets): Numero di righe lette da get_all_values(): {len(list_of_lists) if list_of_lists else 'Nessuna'}")
        if list_of_lists and len(list_of_lists) > 0:
            print(f"--- DEBUG (load_google_sheets): Prima riga letta (header?): {list_of_lists[0]}")
        if list_of_lists and len(list_of_lists) > 1:
            print(f"--- DEBUG (load_google_sheets): Seconda riga letta (primi dati?): {list_of_lists[1][:5]}") # Mostra solo i primi 5 elementi della seconda riga
        # ---> FINE DEBUG <---

        if not list_of_lists or len(list_of_lists) < 2: # Deve esserci almeno una riga di header e una di dati
            print(f"❌ Nessun dato o solo header trovato nel foglio '{worksheet_name}'.")
            return None

        headers = [str(h).strip().replace("[","").replace("]","") for h in list_of_lists[0]]
        df = pd.DataFrame(list_of_lists[1:], columns=headers)

        if df.empty:
            print(f"❌ DataFrame vuoto dopo la creazione da list_of_lists per '{worksheet_name}'.")
            return None

        print("--- DEBUG (load_google_sheets): Prime 5 righe grezze lette ---")
        print(df.head())
        print("--- DEBUG (load_google_sheets): Tipi di dato iniziali ---")
        print(df.dtypes)
        print(f"--- DEBUG (load_google_sheets): Nomi colonne: {df.columns.tolist()} ---")

        date_col_candidates = ['Date', 'date', 'Data', 'data', 'Time', 'Timestamp', 'Giorno', 'giorno', 'datetime']
        date_col_found = None
        for dc_candidate in date_col_candidates:
            for actual_col_name in df.columns:
                if actual_col_name.lower() == dc_candidate.lower():
                    date_col_found = actual_col_name
                    break
            if date_col_found: break

        if not date_col_found:
            print(f"❌ Nessuna colonna data/ora riconosciuta. Colonne: {df.columns.tolist()}")
            return None
        print(f"--- DEBUG (load_google_sheets): Colonna data/ora identificata: '{date_col_found}' ---")

        try:
            # Prova a specificare il formato se sai che è consistente GG/MM/YYYY HH.MM.SS
            # Se l'orario può variare o non essere sempre presente, pd.to_datetime potrebbe essere più flessibile
            # ma il formato con i punti nell'ora è atipico.
            df[date_col_found] = df[date_col_found].astype(str) # Assicurati sia stringa
            # Prima prova un formato comune, poi con i punti nell'ora se necessario
            # Specifica il formato corretto per le tue date/ore GG/MM/YYYY HH.MM.SS
            df['datetime_converted_temp'] = pd.to_datetime(df[date_col_found], format='%d/%m/%Y %H.%M.%S', errors='coerce')

            print(f"--- DEBUG (load_google_sheets): Valori data dopo conversione (primi 5): ---")
            print(df['datetime_converted_temp'].head())

            na_count = df['datetime_converted_temp'].isna().sum()
            print(f"--- DEBUG (load_google_sheets): NaT count: {na_count} su {len(df)} ---")
            if na_count == len(df) and len(df) > 0:
                print(f"❌ TUTTE le date non convertite per '{date_col_found}'. Formato originale esempio: {df[date_col_found].iloc[0]}")
                return None

            df.set_index('datetime_converted_temp', inplace=True)
            df = df.loc[pd.notna(df.index)]
            if df.empty:
                print(f"❌ DataFrame vuoto dopo conversione data e rimozione NaT.")
                return None
            df.sort_index(inplace=True)
        except Exception as e_date:
            print(f"❌ Errore conversione/impostazione data '{date_col_found}': {e_date}")
            return None

        # Conversione colonne numeriche
        cols_to_convert_to_numeric = ['Open', 'High', 'Low', 'Close', 'Volume']
        final_cols_to_keep = []

        for col_name_base in cols_to_convert_to_numeric:
            actual_col_name_in_df = None
            # Cerca varianti case-insensitive del nome colonna
            for potential_name in [col_name_base, col_name_base.lower(), col_name_base.capitalize()]: # Aggiunto .capitalize()
                 if potential_name == 'Close' and 'Chiusura' in df.columns: # Caso specifico per 'Chiusura' se 'Close' non c'è
                     potential_name = 'Chiusura'
                 if potential_name in df.columns:
                    actual_col_name_in_df = potential_name
                    break

            if actual_col_name_in_df:
                try:
                    print(f"--- DEBUG (load_google_sheets): Converto colonna '{actual_col_name_in_df}' (target: {col_name_base}) ---")
                    # Forza a stringa, sostituisci virgola, poi float
                    df[col_name_base] = df[actual_col_name_in_df].astype(str).str.replace(',', '.', regex=False).astype(float)
                    final_cols_to_keep.append(col_name_base)
                    print(f"Primi 5 valori convertiti per '{col_name_base}': {df[col_name_base].head().tolist()}")
                except Exception as e_num:
                    print(f"AVVISO: Errore conversione colonna '{actual_col_name_in_df}' a numerico: {e_num}. Ignorata.")
            elif col_name_base == 'Close': # 'Close' è obbligatoria
                 print(f"❌ Colonna 'Close' (o varianti) non trovata. Colonne: {df.columns.tolist()}")
                 return None

        if 'Close' not in final_cols_to_keep: # Ricontrolla se 'Close' è stata convertita
            print(f"❌ Colonna 'Close' non convertita correttamente o non trovata.")
            return None

        df_final = df[final_cols_to_keep].dropna(subset=['Close'])
        if df_final.empty:
            print(f"❌ DataFrame vuoto dopo dropna su 'Close'.")
            return None

        print(f"✅ Dati caricati e processati da '{worksheet_name}'. Colonne: {df_final.columns.tolist()}")
        print(f"--- DEBUG (load_google_sheets): Prime 5 righe DataFrame finale: ---")
        print(df_final.head())
        return df_final.copy()

    except Exception as e:
        print(f"❌ Errore generale in _load_data_from_google_sheets ('{worksheet_name}'): {e}")
        import traceback; traceback.print_exc()
        return None
        # Gestione colonne opzionali OHLCV
        for original_name, target_name in numeric_cols_map.items():
            if original_name in df.columns and target_name != 'Close': # 'Close' è già gestita
                try:
                    df[target_name] = df[original_name].astype(str).str.replace(',', '.', regex=False).astype(float)
                    if target_name not in cols_to_return_final:
                         cols_to_return_final.append(target_name)
                except Exception as e_col:
                    print(f"AVVISO: Errore nella conversione della colonna '{original_name}' a numerico: {e_col}. Verrà ignorata.")
                    if target_name in cols_to_return_final: # Rimuovi se la conversione fallisce
                        cols_to_return_final.remove(target_name)

        # Filtra per le colonne effettivamente convertite e presenti
        df_final = df[[col for col in ['Open', 'High', 'Low', 'Close', 'Volume'] if col in cols_to_return_final and col in df.columns]]

        # Assicurati che 'Close' non abbia NaN dopo la conversione
        df_final = df_final.dropna(subset=['Close'])
        if df_final.empty:
            print(f"❌ DataFrame vuoto dopo la conversione numerica e rimozione NaN per 'Close' in '{worksheet_name}'.")
            return None

        print(f"✅ Dati caricati e processati da '{worksheet_name}'. Colonne restituite: {df_final.columns.tolist()}")
        return df_final.copy() # Restituisci una copia per evitare SettingWithCopyWarning

    except Exception as e:
        print(f"❌ Errore generale durante il caricamento da Google Sheets ('{worksheet_name}'): {e}")
        import traceback; traceback.print_exc()
        return None

In [ ]:
def _load_data_from_fmp(symbol, api_key, start_date, end_date):
    print(f"Caricamento dati da FMP per {symbol}...")
    if not api_key: print("API Key FMP mancante."); return None
    url = f"https://financialmodelingprep.com/api/v3/historical-price-full/{symbol}?from={start_date}&to={end_date}&apikey={api_key}"
    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        data = response.json()
        if 'historical' in data and data['historical']:
            df = pd.DataFrame(data['historical'])
            df['date'] = pd.to_datetime(df['date'])
            df = df.set_index('date').sort_index()
            df = df.rename(columns={'close': 'Close', 'open': 'Open', 'high': 'High', 'low': 'Low', 'volume': 'Volume'})
            required_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
            df = df[[col for col in required_cols if col in df.columns]] # Select only available required columns
            print(f"Dati caricati da FMP per {symbol}.")
            return df
        else: print(f"Nessun dato 'historical' da FMP per {symbol}. Risposta: {data}"); return None
    except requests.exceptions.ReadTimeout:
        print(f"Timeout durante la richiesta a FMP per {symbol}.")
        return None
    except Exception as e: print(f"Errore FMP per {symbol}: {e}"); return None

def _load_data_from_alphavantage(symbol, api_key, start_date, end_date):
    print(f"Caricamento dati da Alpha Vantage per {symbol}...")
    if not api_key: print("API Key Alpha Vantage mancante."); return None
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&outputsize=full&apikey={api_key}"
    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        data = response.json()
        if "Time Series (Daily)" in data:
            df = pd.DataFrame.from_dict(data["Time Series (Daily)"], orient='index')
            df.index = pd.to_datetime(df.index)
            df = df.sort_index()
            df = df.rename(columns={'1. open': 'Open', '2. high': 'High', '3. low': 'Low', '4. close': 'Close', '5. adjusted close': 'Adjusted_Close', '6. volume': 'Volume'})
            # Alpha Vantage returns data in descending order by default, already sorted by sort_index()
            df = df.loc[(df.index >= pd.to_datetime(start_date)) & (df.index <= pd.to_datetime(end_date))]
            required_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
            # Prefer 'Close' over 'Adjusted_Close' if both exist and 'Close' was requested, or use 'Adjusted_Close' if 'Close' is not present
            if 'Adjusted_Close' in df.columns and 'Close' not in df.columns:
                df.rename(columns={'Adjusted_Close': 'Close'}, inplace=True)
            df = df[[col for col in required_cols if col in df.columns]]
            print(f"Dati caricati da Alpha Vantage per {symbol}.")
            return df
        elif "Error Message" in data: print(f"Errore Alpha Vantage per {symbol}: {data['Error Message']}"); return None
        elif "Information" in data: print(f"Informazione da Alpha Vantage (potrebbe essere un limite API): {data['Information']}"); return None
        else: print(f"Formato dati Alpha Vantage inatteso per {symbol}. Risposta: {data}"); return None
    except requests.exceptions.ReadTimeout:
        print(f"Timeout durante la richiesta a Alpha Vantage per {symbol}.")
        return None
    except Exception as e: print(f"Errore Alpha Vantage per {symbol}: {e}"); return None

def _load_data_from_twelvedata(symbol, api_key, start_date, end_date):
    print(f"Caricamento dati da Twelve Data per {symbol}...")
    if not api_key: print("API Key Twelve Data mancante."); return None
    url = f"https://api.twelvedata.com/time_series?symbol={symbol}&interval=1day&start_date={start_date}&end_date={end_date}&apikey={api_key}&outputsize=5000"
    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        data = response.json()
        if "values" in data and data["values"]:
            df = pd.DataFrame(data["values"])
            df['datetime'] = pd.to_datetime(df['datetime'])
            df = df.set_index('datetime').sort_index()
            df = df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'})
            numeric_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
            for col in numeric_cols: df[col] = pd.to_numeric(df[col], errors='coerce')
            df = df[[col for col in numeric_cols if col in df.columns]]
            print(f"Dati caricati da Twelve Data per {symbol}.")
            return df
        elif data.get("status") == "error" or data.get("code", 200) != 200 :
            print(f"Errore da Twelve Data per {symbol} ({data.get('code')}): {data.get('message', 'Errore sconosciuto')}")
            return None
        else: print(f"Formato dati Twelve Data inatteso o vuoto per {symbol}. Risposta: {data}"); return None
    except requests.exceptions.ReadTimeout:
        print(f"Timeout durante la richiesta a Twelve Data per {symbol}.")
        return None
    except Exception as e: print(f"Errore Twelve Data per {symbol}: {e}"); return None

def _load_data_from_tiingo(symbol, api_key, start_date, end_date):
    print(f"Caricamento dati da Tiingo per {symbol}...")
    if not api_key: print("API Key Tiingo mancante."); return None
    headers = {'Authorization': f'Token {api_key}'}
    url = f"https://api.tiingo.com/tiingo/daily/{symbol}/prices?startDate={start_date}&endDate={end_date}&format=json"
    try:
        response = requests.get(url, headers=headers, timeout=30)
        response.raise_for_status()
        data = response.json()
        if data and isinstance(data, list):
            df = pd.DataFrame(data)
            df['date'] = pd.to_datetime(df['date'])
            df = df.set_index('date').sort_index()
            df = df.rename(columns={'close': 'Close', 'open': 'Open', 'high': 'High', 'low': 'Low', 'volume': 'Volume', 'adjClose': 'Adjusted_Close'})
            required_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
            if 'Adjusted_Close' in df.columns and 'Close' not in df.columns:
                 df.rename(columns={'Adjusted_Close': 'Close'}, inplace=True)
            df = df[[col for col in required_cols if col in df.columns]]
            print(f"Dati caricati da Tiingo per {symbol}.")
            return df
        elif isinstance(data, dict) and 'detail' in data:
            print(f"Errore da Tiingo per {symbol}: {data['detail']}")
            return None
        else: print(f"Nessun dato o formato inatteso da Tiingo per {symbol}. Risposta: {data}"); return None
    except requests.exceptions.ReadTimeout:
        print(f"Timeout durante la richiesta a Tiingo per {symbol}.")
        return None
    except Exception as e: print(f"Errore Tiingo per {symbol}: {e}"); return None

def _load_data_from_eod(symbol, api_key, start_date, end_date):
    print(f"Caricamento dati da EOD Historical Data per {symbol}...")
    if not api_key: print("API Key EOD mancante."); return None
    symbol_eod = f"{symbol}.US" if '.' not in symbol else symbol # Assume US exchange if not specified
    url = f"https://eodhistoricaldata.com/api/eod/{symbol_eod}?from={start_date}&to={end_date}&api_token={api_key}&fmt=json&period=d"
    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        data = response.json()
        if data and isinstance(data, list):
            df = pd.DataFrame(data)
            df['date'] = pd.to_datetime(df['date'])
            df = df.set_index('date').sort_index()
            # EOD uses 'adjusted_close'. We need 'Close'.
            df = df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close':'Raw_Close', 'adjusted_close': 'Close', 'volume': 'Volume'})
            required_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
            df = df[[col for col in required_cols if col in df.columns]]
            print(f"Dati caricati da EOD per {symbol}.")
            return df
        elif isinstance(data, str) and ('Error' in data or 'error' in data):
             print(f"Errore da EOD per {symbol}: {data}")
             return None
        else: print(f"Nessun dato valido o formato inatteso da EOD per {symbol}. Risposta: {data}"); return None
    except requests.exceptions.ReadTimeout:
        print(f"Timeout durante la richiesta a EOD Historical Data per {symbol}.")
        return None
    except Exception as e: print(f"Errore EOD per {symbol}: {e}"); return None


In [ ]:
# NELLA CELLA 12 - Modifica la funzione carica_dati_da_file

import pandas as pd
import io

def carica_dati_da_file(uploaded_file_content_stream):
    """
    Carica i dati da un file CSV o Excel (passato come stream) e formatta il DataFrame.
    Ottimizzata per un file CSV/Excel con header alla prima riga, date standard
    e punto come separatore decimale.
    """
    try:
        file_name = getattr(uploaded_file_content_stream, 'name', 'untitled.csv').lower()
        print(f"Tentativo di caricamento del file (formato atteso pulito): {file_name}")

        df = None
        is_excel = file_name.endswith(('.xls', '.xlsx'))
        is_csv = file_name.endswith('.csv')

        if is_csv:
            uploaded_file_content_stream.seek(0)
            df = pd.read_csv(uploaded_file_content_stream, header=0, sep=',', decimal='.') # Assume virgola come separatore e punto come decimale
            print("--- DEBUG (carica_dati_da_file): Letto CSV ---")
        elif is_excel:
            uploaded_file_content_stream.seek(0)
            df = pd.read_excel(uploaded_file_content_stream, header=0)
            print("--- DEBUG (carica_dati_da_file): Letto file Excel ---")
        else:
            raise ValueError("Formato file non supportato.")

        if df is None or df.empty:
            print("❌ DataFrame vuoto dopo la lettura iniziale del file.")
            return pd.DataFrame()

        print("--- DEBUG (carica_dati_da_file): Prime 5 righe lette dal file ---")
        print(df.head())
        print("--- DEBUG (carica_dati_da_file): Tipi di dato iniziali ---")
        print(df.dtypes)

        df.columns = [str(col).strip().lower() for col in df.columns]
        print(f"--- DEBUG (carica_dati_da_file): Nomi colonne puliti: {df.columns.tolist()} ---")

        # --- MODIFICA QUI I NOMI DELLE COLONNE ORIGINALI ATTESE ---
        date_col_original_name = 'date' # Ora ci aspettiamo 'date' dopo .lower()
        close_col_original_name = 'close' # Ora ci aspettiamo 'close' dopo .lower(), invece di 'nav per share'
        # ---------------------------------------------------------

        if date_col_original_name not in df.columns:
            # Prova a cercare alternative comuni se 'date' non è presente
            alt_date_names = ['data', 'time', 'timestamp', 'giorno']
            found_alt_date = False
            for alt_name in alt_date_names:
                if alt_name in df.columns:
                    date_col_original_name = alt_name
                    print(f"AVVISO: Colonna 'date' non trovata, usando '{alt_name}' come colonna data.")
                    found_alt_date = True
                    break
            if not found_alt_date:
                 raise ValueError(f"Colonna data ('{date_col_original_name}' o alternative) non trovata. Colonne: {df.columns.tolist()}")

        # Non c'è bisogno di rinominare 'Close' se si chiama già 'close' dopo .lower()
        # Ma se il nome originale fosse stato 'NAV per share' e l'avessimo voluto standardizzare a 'Close' (maiuscolo),
        # la rinomina andrebbe fatta qui. Dato che ora è 'close', va bene.
        # Se la colonna nel file fosse "NAV per share", dopo .lower() diventerebbe "nav per share".
        # Se vogliamo che il DataFrame finale abbia una colonna 'Close' (maiuscola), dobbiamo rinominarla.
        # Per coerenza con il resto del codice che si aspetta 'Close' (maiuscolo):
        if close_col_original_name in df.columns and 'Close' not in df.columns:
            df.rename(columns={close_col_original_name: 'Close'}, inplace=True)
            print(f"--- DEBUG (carica_dati_da_file): Colonna '{close_col_original_name}' rinominata in 'Close' ---")
            final_close_col_name = 'Close'
        elif 'Close' in df.columns: # Se si chiama già 'Close' (maiuscolo) nel file
            final_close_col_name = 'Close'
        elif 'close' in df.columns: # Se si chiama 'close' (minuscolo) nel file
             final_close_col_name = 'close' # Useremo questa per la conversione, poi la selezioneremo come 'Close'
        else:
            raise ValueError(f"Colonna prezzo ('{close_col_original_name}' o 'Close' o 'close') non trovata. Colonne: {df.columns.tolist()}")

        # Conversione colonna Data
        try:
            # Pandas è bravo a inferire formati data comuni, ma se serve specificare:
            # df[date_col_original_name] = pd.to_datetime(df[date_col_original_name], format='%d/%m/%Y', errors='coerce')
            df[date_col_original_name] = pd.to_datetime(df[date_col_original_name], errors='coerce')
            df.set_index(date_col_original_name, inplace=True)
            df = df.loc[pd.notna(df.index)]
            if df.empty: print("❌ DataFrame vuoto dopo conversione/rimozione NaT date."); return pd.DataFrame()
            df.sort_index(inplace=True)
        except Exception as e_date:
            print(f"❌ Errore conversione data '{date_col_original_name}': {e_date}"); return pd.DataFrame()

        # Conversione colonna Close a numerico
        try:
            # Dato che l'hai pulito per usare il punto come decimale, una conversione diretta dovrebbe funzionare
            df[final_close_col_name] = pd.to_numeric(df[final_close_col_name], errors='coerce')
        except Exception as e_close:
            print(f"❌ Errore conversione '{final_close_col_name}' a numerico: {e_close}"); return pd.DataFrame()

        # Seleziona solo la colonna 'Close' (assicurati che il nome sia 'Close' maiuscolo se necessario)
        # Se final_close_col_name era 'close', e vuoi 'Close', rinomina prima di selezionare
        if final_close_col_name == 'close' and 'Close' not in df.columns:
            df.rename(columns={'close':'Close'}, inplace=True)

        cols_to_return = ['Close']
        # Aggiungi qui la logica per Open, High, Low, Volume se presenti nel CSV pulito e se ti servono
        # Esempio per 'Open', assumendo che il file pulito abbia la colonna 'Open' (o 'open'):
        # potential_ohlv_names = {'open_orig': 'Open', 'high_orig': 'High', 'low_orig': 'Low', 'volume_orig': 'Volume'}
        # for orig_name_key, target_name in potential_ohlv_names.items():
        #     # Trova il nome effettivo (case-insensitive)
        #     actual_name_in_df = None
        #     for col_in_df in df.columns:
        #         if col_in_df.lower() == orig_name_key.replace('_orig','').lower(): # es. 'open'
        #             actual_name_in_df = col_in_df
        #             break
        #     if actual_name_in_df:
        #         df[target_name] = pd.to_numeric(df[actual_name_in_df], errors='coerce')
        #         if target_name not in cols_to_return: cols_to_return.append(target_name)

        # Assicurati che le colonne selezionate esistano effettivamente nel DataFrame
        final_cols_to_keep_in_df = [col for col in cols_to_return if col in df.columns]
        if not final_cols_to_keep_in_df:
             print(f"❌ Nessuna colonna valida da restituire ({cols_to_return}) trovata in df.columns: {df.columns.tolist()}")
             return pd.DataFrame()

        final_df = df[final_cols_to_keep_in_df].dropna(subset=['Close'])

        if final_df.empty:
            print(f"❌ DataFrame finale vuoto."); return pd.DataFrame()

        print(f"✅ File letto e processato. Colonne restituite: {final_df.columns.tolist()}")
        print(f"--- DEBUG (carica_dati_da_file): Prime 5 righe DataFrame finale: ---")
        print(final_df.head())
        return final_df

    except Exception as e:
        print(f"Errore dettagliato nel caricamento del file '{getattr(uploaded_file_content_stream, 'name', 'N/A')}': {e}")
        import traceback; traceback.print_exc()
        return pd.DataFrame()

## 5. Funzioni Utilità per Analisi e Plotting

In [ ]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import seaborn as sns
from IPython.display import display, HTML


# -----------------------------------------------------
# Utility per assicurare che i rendimenti siano in scala decimale
# -----------------------------------------------------

# --- FUNZIONI UTILITY PER ANALISI E PLOTTING (da cella 10 originale - CORRETTE/ADATTATE) ---
def ensure_decimal_returns(series_returns):
    """Assicura che i rendimenti siano in formato decimale (es. 0.01 per 1%)."""
    if series_returns.empty: return series_returns
    # Se la maggior parte dei valori assoluti è piccola, ma il max è > 1 (es. 1.5 per 1.5%)
    # è probabile che siano già percentuali, ma con qualche outlier.
    # Un criterio più robusto potrebbe controllare la mediana o i quantili.
    # Per ora, un semplice controllo sul massimo assoluto.
    if series_returns.abs().max() > 1.0 and series_returns.abs().median() > 0.0001 : # Evita di dividere rendimenti già molto piccoli
        print("INFO: Rilevati rendimenti > 1, si assume siano in formato percentuale e verranno divisi per 100.")
        return series_returns / 100.0
    return series_returns


# -----------------------------------------------------
# Simulazione Monte Carlo basata sul modello GARCH fittato
# -----------------------------------------------------

def simulazione_monte_carlo(modello_fit, n_simulazioni=1000, giorni=30):
    """Simula i percorsi dei rendimenti futuri da un modello GARCH fittato."""
    print(
        f"Avvio simulazione Monte Carlo: {modello_fit.model.name} - "
        f"{modello_fit.model.volatility.name} - {modello_fit.model.distribution.name}"
    )
    try:
        forecast_results = modello_fit.forecast(
            horizon=giorni, method="simulation", simulations=n_simulazioni
        )
        simulated_returns = forecast_results.simulations.values[0].T  # (giorni, N)
        print(
            f"Simulazione completata. Shape dei rendimenti simulati: {simulated_returns.shape}"
        )
        return simulated_returns
    except Exception as e:
        print(
            f"Errore durante la simulazione Monte Carlo (metodo 'forecast'): {e}\n"
            "Tentativo con simulazione manuale…"
        )
        # ----------------------
        # Fallback manuale (semplice GJR‑GARCH(1,1))
        # ----------------------
        params = modello_fit.params
        omega = params.get("omega", 0)
        alpha_p = params.get("alpha[1]", 0)
        beta_p = params.get("beta[1]", 0)
        gamma_p = params.get("gamma[1]", 0)  # per GJR

        last_resid = modello_fit.resid.iloc[-1] if len(modello_fit.resid) else 0.0
        last_sigma2 = (
            modello_fit.conditional_volatility.iloc[-1] ** 2
            if len(modello_fit.conditional_volatility)
            else omega / max(1e-9, (1 - alpha_p - beta_p - gamma_p / 2))
        )
        last_sigma2 = max(last_sigma2, 1e-9)

        sim_returns_manual = np.zeros((giorni, n_simulazioni))
        std_resid_dist = modello_fit.std_resid.dropna()

        for i in range(n_simulazioni):
            current_sigma2 = last_sigma2
            current_resid = last_resid
            innovations = (
                np.random.choice(std_resid_dist, size=giorni, replace=True)
                if not std_resid_dist.empty
                else np.random.normal(size=giorni)
            )
            for t_step in range(giorni):
                gjr_term = gamma_p * current_resid ** 2 * (current_resid < 0)
                current_sigma2 = (
                    omega + alpha_p * current_resid ** 2 + beta_p * current_sigma2 + gjr_term
                )
                current_sigma2 = max(current_sigma2, 1e-9)
                sim_returns_manual[t_step, i] = np.sqrt(current_sigma2) * innovations[t_step]
                current_resid = sim_returns_manual[t_step, i]
        return sim_returns_manual

# -----------------------------------------------------
# Fan chart Plotly – ora i rendimenti sono convertiti in % solo per il grafico
# -----------------------------------------------------
def fan_chart_plotly(simulated_returns_decimal, titolo="Simulazioni Monte Carlo Rendimenti", quantili=None, livello_var_decimal=0.05, clear: bool = False):
    if simulated_returns_decimal is None or simulated_returns_decimal.size == 0:
        print("Nessuna simulazione da plottare.")
        return None

    if quantili is None:
        quantili = [1, 5, 25, 50, 75, 95, 99]

    simulazioni_pct = simulated_returns_decimal * 100.0  # %

    giorni_axis = np.arange(simulazioni_pct.shape[0])

    var_quantile_display = livello_var_decimal * 100
    quantili = sorted(set(quantili + [var_quantile_display, 50]))

    # Calcolo percentili CORRETTO: axis=1 perché (giorni, simulazioni)
    percentili_data = np.percentile(simulazioni_pct, quantili, axis=1)
    # Ora percentili_data.shape = (num_quantili, num_giorni)

    fig = go.Figure()

    percentiles_map = {q: percentili_data[i, :] for i, q in enumerate(quantili)}

    num_bands = (len(quantili) - 1) // 2
    for i in range(num_bands):
        lower_q, upper_q = quantili[i], quantili[-(i + 1)]
        if lower_q >= upper_q:
            continue
        fig.add_trace(go.Scatter(
            x=np.concatenate([giorni_axis, giorni_axis[::-1]]),
            y=np.concatenate([percentiles_map[upper_q], percentiles_map[lower_q][::-1]]),
            fill="toself",
            fillcolor=f"rgba(0, 100, 200, {0.1 + 0.6 * (i / max(1, num_bands))})",
            line=dict(width=0),
            name=f"{lower_q:.0f}-{upper_q:.0f}% perc.",
            hoverinfo="skip"
        ))

    # Linea mediana (50%)
    fig.add_trace(go.Scatter(
        x=giorni_axis,
        y=percentiles_map[50],
        mode='lines',
        line=dict(color='black', width=2),
        name='Mediana (50%)'
    ))

    ultimo_giorno_sims_pct = simulazioni_pct[-1, :]
    var_val_calc_pct = np.percentile(ultimo_giorno_sims_pct, var_quantile_display)
    cvar_val_calc_pct = ultimo_giorno_sims_pct[ultimo_giorno_sims_pct <= var_val_calc_pct].mean()

    fig.add_trace(go.Scatter(
        x=[giorni_axis[-1]],
        y=[var_val_calc_pct],
        mode='markers',
        marker=dict(color='red', size=10, symbol='line-ew-open'),
        name=f'VaR ({var_quantile_display:.0f}%) Ult.G: {var_val_calc_pct:.2f}%'
    ))

    fig.add_trace(go.Scatter(
        x=[giorni_axis[-1]],
        y=[cvar_val_calc_pct],
        mode='markers',
        marker=dict(color='darkred', size=10, symbol='line-ew-open'),
        name=f'CVaR ({var_quantile_display:.0f}%) Ult.G: {cvar_val_calc_pct:.2f}%'
    ))

    fig.update_layout(
        title=titolo,
        xaxis_title="Giorni di Previsione",
        yaxis_title="Rendimento Simulato (%)",
        template="plotly_white",
        hovermode="x unified"
    )


    #show_plotly(fig, clear=clear)
    return fig

# AGGIUNGI QUESTA NUOVA FUNZIONE NELLA CELLA DELLE FUNZIONI HELPER

def mostra_confronto_volatilita(df_dati, fitted_model, label_serie="", clear=False):
    """
    Crea un grafico che confronta la volatilità storica (realizzata) con quella
    prevista dal modello GARCH.
    """
    try:
        print(f"--- 📈 Creazione grafico confronto volatilità per {label_serie} ---")

        # 1. Calcola la volatilità storica (realizzata) con una finestra mobile
        # Usiamo una finestra di 21 giorni (circa un mese di trading)
        rendimenti_decimali = df_dati['returns'].dropna()
        vol_realizzata_pct = rendimenti_decimali.rolling(window=21).std() * np.sqrt(252) * 100

        # 2. Estrai la volatilità prevista dal modello GARCH e la annualizzi
        # Il modello è fittato su rendimenti in punti %, quindi la volatilità è già in %
        vol_prevista_garch_pct = fitted_model.conditional_volatility * np.sqrt(252)

        # 3. Crea il grafico
        fig = go.Figure()

        # Aggiungi la traccia della volatilità storica (realizzata)
        fig.add_trace(go.Scatter(
            x=vol_realizzata_pct.index,
            y=vol_realizzata_pct,
            mode='lines',
            name='Volatilità Storica (Rolling 21gg)',
            line=dict(color='grey', width=1.5)
        ))

        # Aggiungi la traccia della volatilità prevista dal modello GARCH
        fig.add_trace(go.Scatter(
            x=vol_prevista_garch_pct.index,
            y=vol_prevista_garch_pct,
            mode='lines',
            name='Volatilità Prevista (GARCH)',
            line=dict(color='royalblue', width=2)
        ))

        # 4. Layout e visualizzazione
        fig.update_layout(
            title=f'Confronto Volatilità Storica vs. Prevista - {label_serie}',
            xaxis_title='Data',
            yaxis_title='Volatilità Annualizzata (%)',
            template='plotly_white',
            legend=dict(x=0.01, y=0.98, bgcolor='rgba(255,255,255,0.6)')
        )

        #show_plotly(fig, clear=clear)
        return fig

    except Exception as e:
        print(f"❌ Errore durante la creazione del grafico di confronto volatilità: {e}")
        return None

# AGGIUNGI QUESTA NUOVA FUNZIONE NELLA CELLA DELLE FUNZIONI HELPER

def mostra_grafici_acf_pacf(returns_series, label_serie="", lags=40):
    """
    Crea e visualizza i grafici ACF e PACF per i rendimenti e
    i rendimenti al quadrato utilizzando matplotlib.
    """
    # Assicurati che la serie di input non abbia valori mancanti
    returns_series = returns_series.dropna()

    print(f"\n--- 🔬 Analisi di Autocorrelazione (ACF/PACF) per {label_serie} ---")

    # Creiamo una figura matplotlib con 4 sotto-grafici (2 righe, 2 colonne)
    fig, axes = plt.subplots(2, 2, figsize=(16, 8))
    fig.suptitle(f'Analisi di Autocorrelazione per {label_serie}', fontsize=16)

    # --- Riga 1: Rendimenti ---
    # Ci aspettiamo poche barre fuori dalla zona blu (nessuna "memoria" sui rendimenti)
    plot_acf(returns_series, lags=lags, ax=axes[0, 0], title='ACF Rendimenti')
    axes[0, 0].set_xlabel('Lag')
    axes[0, 0].set_ylabel('ACF')

    plot_pacf(returns_series, lags=lags, ax=axes[0, 1], title='PACF Rendimenti')
    axes[0, 1].set_xlabel('Lag')
    axes[0, 1].set_ylabel('PACF')

    # --- Riga 2: Rendimenti al Quadrato (per testare gli effetti ARCH) ---
    # Ci aspettiamo molte barre fuori dalla zona blu (segno di volatility clustering)
    plot_acf(returns_series**2, lags=lags, ax=axes[1, 0], title='ACF Rendimenti al Quadrato')
    axes[1, 0].set_xlabel('Lag')
    axes[1, 0].set_ylabel('ACF')

    plot_pacf(returns_series**2, lags=lags, ax=axes[1, 1], title='PACF Rendimenti al Quadrato')
    axes[1, 1].set_xlabel('Lag')
    axes[1, 1].set_ylabel('PACF')

    # Migliora la spaziatura e mostra il grafico
    plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Aggiusta per il titolo principale

    # plt.show() disegnerà il grafico statico nell'output della cella
    plt.show()

# -----------------------------------------------------
# Volatilità prevista – conversione in percentuale sul plot
# -----------------------------------------------------


def mostra_volatilita_prevista(fitted_model, giorni_forecast=30, title_suffix="", start_date_forecast=None, out_area=None, clear: bool = False):
    try:
        forecast = fitted_model.forecast(horizon=giorni_forecast, method="simulation", simulations=1000)
        sim_var = forecast.simulations.residual_variances

        if sim_var.ndim == 3 and sim_var.shape[0] == 1:
            sim_var = sim_var[0]
        if sim_var.shape[0] != giorni_forecast and sim_var.shape[1] == giorni_forecast:
            sim_var = sim_var.T

        simulated_volatility_decimal = np.sqrt(sim_var)
        simulated_volatility_pct = simulated_volatility_decimal * 100

        percentili_plot = [5, 25, 50, 75, 95]
        vol_percentiles_data_pct = np.percentile(simulated_volatility_pct, percentili_plot, axis=1)

        if start_date_forecast:
            forecast_dates_axis = pd.date_range(start=start_date_forecast + pd.offsets.BDay(1),
                                                periods=giorni_forecast, freq="B")
            x_label_plot = "Data di Previsione"
        else:
            forecast_dates_axis = np.arange(1, giorni_forecast + 1)
            x_label_plot = "Giorni di Previsione"

        fig = go.Figure()

        fig.add_trace(go.Scatter(
            x=np.concatenate([forecast_dates_axis, forecast_dates_axis[::-1]]),
            y=np.concatenate([vol_percentiles_data_pct[4], vol_percentiles_data_pct[0][::-1]]),
            fill='toself', fillcolor='rgba(100,149,237,0.2)', line=dict(width=0),
            name='5-95 percentile'
        ))

        fig.add_trace(go.Scatter(
            x=np.concatenate([forecast_dates_axis, forecast_dates_axis[::-1]]),
            y=np.concatenate([vol_percentiles_data_pct[3], vol_percentiles_data_pct[1][::-1]]),
            fill='toself', fillcolor='rgba(100,149,237,0.4)', line=dict(width=0),
            name='25-75 percentile (IQR)'
        ))

        fig.add_trace(go.Scatter(
            x=forecast_dates_axis,
            y=vol_percentiles_data_pct[2],
            mode='lines', line=dict(color='blue', width=2),
            name='Volatilità Mediana Prevista (%)'
        ))

        fig.update_layout(
            title=f"Previsione Volatilità Giornaliera (%) ({title_suffix}) - {giorni_forecast} giorni",
            xaxis_title=x_label_plot,
            yaxis_title="Volatilità Giornaliera Prevista (%)",
            template='plotly_white',
            hovermode='x unified'
        )



        #show_plotly(fig, clear=clear)
        return fig


    except Exception as e_mvp:
        print(f"Errore in mostra_volatilita_prevista (simulation): {e_mvp}")
        try:
            forecast_puntuale = fitted_model.forecast(horizon=giorni_forecast)
            vol_prevista_pct = np.sqrt(forecast_puntuale.variance.iloc[-1].values) * 100.0

            if start_date_forecast:
                forecast_dates_axis_fb = pd.date_range(start=start_date_forecast + pd.offsets.BDay(1), periods=giorni_forecast, freq="B")
            else:
                forecast_dates_axis_fb = np.arange(1, giorni_forecast + 1)

            fig_fb = go.Figure()
            fig_fb.add_trace(go.Scatter(
                x=forecast_dates_axis_fb,
                y=vol_prevista_pct,
                mode='lines+markers',
                line=dict(color='blue'),
                name='Volatilità Prevista (Fallback puntuale)'
            ))

            fig_fb.update_layout(
                title=f"Volatilità Giorn. Prevista (Puntuale - Fallback) ({title_suffix}) - {giorni_forecast} gg",
                xaxis_title=x_label_plot,
                yaxis_title="Volatilità Prevista (%)",
                template='plotly_white',
                hovermode='x unified'
            )

            #show_plotly(fig_fb, clear=clear)
            return fig_fb # CORREZIONE: Restituisce la figura di fallback
        except Exception as e_fb:
            print(f"Errore anche nel fallback di mostra_volatilita_prevista: {e_fb}")
            return None # CORREZIONE: Restituisce None se anche il fallback fallisce

# -----------------------------------------------------
# VaR/CVaR storico (in decimali) – le percentuali le converti fuori se vuoi grafici in %
# -----------------------------------------------------

def calcola_var_cvar_storico(returns_decimal, alpha_decimal=0.05):
    # (Codice da cella 10 originale - prende rendimenti decimali e restituisce VaR/CVaR decimali)
    if returns_decimal.empty: return np.nan, np.nan
    var_decimal = np.percentile(returns_decimal, alpha_decimal * 100) # alpha_decimal es. 0.05 -> percentile 5
    cvar_decimal = returns_decimal[returns_decimal <= var_decimal].mean()
    return var_decimal, cvar_decimal


# -----------------------------------------------------
# KDE – anche qui, converti in % solo per il grafico
# -----------------------------------------------------

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.stats import gaussian_kde


def mostra_kde_var_cvar(returns_decimal, var_decimal, cvar_decimal, title="Distribuzione dei Rendimenti Giornalieri", clear: bool = False):
    if returns_decimal is None or returns_decimal.empty:
        print("Dati rendimenti non disponibili.")
        return

    returns_pct = returns_decimal * 100.0
    var_pct = var_decimal * 100.0 if not np.isnan(var_decimal) else np.nan
    cvar_pct = cvar_decimal * 100.0 if not np.isnan(cvar_decimal) else np.nan

    kde = gaussian_kde(returns_pct)
    x_vals = np.linspace(returns_pct.min() - 1, returns_pct.max() + 1, 1000)
    y_vals = kde(x_vals)

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=x_vals,
        y=y_vals,
        mode='lines',
        name='Distribuzione KDE',
        fill='tozeroy',
        line=dict(color='skyblue')
    ))

    if not np.isnan(var_pct):
        y_var = float(kde([var_pct]))
        fig.add_trace(go.Scatter(
            x=[var_pct, var_pct],
            y=[0, y_var],
            mode='lines',
            name=f"VaR Storico ({var_pct:.2f}%)",
            line=dict(color='red', dash='dash')
        ))

    if not np.isnan(cvar_pct):
        y_cvar = float(kde([cvar_pct]))
        fig.add_trace(go.Scatter(
            x=[cvar_pct, cvar_pct],
            y=[0, y_cvar],
            mode='lines',
            name=f"CVaR Storico ({cvar_pct:.2f}%)",
            line=dict(color='darkred', dash='dot')
        ))

    fig.update_layout(
        title=title,
        xaxis_title="Rendimenti Giornalieri (%)",
        yaxis_title="Densità Stimata",
        template="plotly_white",
        hovermode="x unified",
        height=500
    )

    #show_plotly(fig, clear=clear)
    return fig




# -----------------------------------------------------
# Esportazione risultati – ora opzionale salvataggio figure, senza NameError
# -----------------------------------------------------


def esporta_risultati(fitted_model, returns_data_decimal, series_label, export_format, base_filename, forecast_horizon):
    # (Codice da cella 10 originale - CORRETTO, rimosso return fig)
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    safe_base = "".join(c if c.isalnum() else "_" for c in base_filename)
    safe_series = "".join(c if c.isalnum() else "_" for c in series_label).replace("__", "_")
    prefix = f"{safe_base}_{safe_series}_{timestamp}"
    print(f"\nInizio esportazione per {series_label} (prefisso: {prefix})")
    try:
        if export_format.lower() == "csv":
            if returns_data_decimal is not None and not returns_data_decimal.empty:
                (returns_data_decimal * 100.0).to_csv(f"{prefix}_returns_pct.csv", header=['Returns (%)']) # Esporta come %
                print(f"  Rendimenti (%) esportati: {prefix}_returns_pct.csv")
            if fitted_model is not None:
                with open(f"{prefix}_garch_summary.txt", "w") as f: f.write(fitted_model.summary().as_text())
                print(f"  Sommario GARCH: {prefix}_garch_summary.txt")
                try:
                    forecast = fitted_model.forecast(horizon=forecast_horizon)
                    (np.sqrt(forecast.variance) * 100.0).to_csv(f"{prefix}_forecast_vol_pct.csv") # Varianza -> Vol %
                    print(f"  Volatilità prevista (%): {prefix}_forecast_vol_pct.csv")
                except Exception as e_fc: print(f"  Errore esportazione volatilità prevista: {e_fc}")
        elif export_format.lower() == "excel":
            excel_filename = f"{prefix}_results.xlsx"
            with pd.ExcelWriter(excel_filename, engine="openpyxl") as writer:
                if returns_data_decimal is not None and not returns_data_decimal.empty:
                    (returns_data_decimal * 100.0).to_excel(writer, sheet_name=f"{safe_series}_Returns_Pct", header=['Returns (%)'])
                if fitted_model is not None:
                    txt = fitted_model.summary().as_text().split("\\n")
                    pd.DataFrame(txt).to_excel(writer, sheet_name=f"{safe_series}_GARCH_Summary", index=False, header=False)
                    try:
                        forecast = fitted_model.forecast(horizon=forecast_horizon)
                        (np.sqrt(forecast.variance) * 100.0).to_excel(writer, sheet_name=f"{safe_series}_ForecastVol_Pct")
                    except Exception as e_fcxl: print(f"  Errore varianza in Excel: {e_fcxl}")
            print(f"Risultati esportati in {excel_filename}")
        else: print("Formato di esportazione non supportato (usa 'csv' o 'excel').")
    except Exception as e_exp: print(f"Errore durante l'esportazione: {e_exp}"); import traceback; traceback.print_exc()
    # Nessun return fig qui

## 6. Definizione Widgets Interfaccia Utente

In [ ]:
# Widget per scegliere tipo di analisi
tipo_analisi = widgets.ToggleButtons(
    options=['Analisi singola', 'Confronto'], description='Tipo Analisi:', style={'description_width': 'initial'}
)

# Selettori delle fonti
fonte_options = ['Alpha Vantage', 'FMP', 'Tiingo', 'EOD', 'Twelve Data', 'Google Sheets', 'File Excel/CSV']
fonte1_dropdown = widgets.Dropdown(options=fonte_options, description='Fonte Serie 1:', style={'description_width': 'initial'})
fonte2_dropdown = widgets.Dropdown(options=fonte_options, description='Fonte Serie 2:', style={'description_width': 'initial'})

# Input per ticker o URL (per Google Sheets, per esempio)
ticker1_input = widgets.Text(description='Ticker/ID Serie 1:', placeholder='Es. AAPL o ID Google Sheet')
ticker2_input = widgets.Text(description='Ticker/ID Serie 2:', placeholder='Es. MSFT o ID Google Sheet')

# ---> INIZIO BLOCCO WIDGET PER GOOGLE SHEETS <---
gsheet_id_s1_input = widgets.Text(
    value='1ukhNTZC0eJpfFF4V1_S9ZN19jM5xqzjhHx2V3PD8leo', # Il tuo ID Spreadsheet di default per Serie 1
    placeholder='Inserisci ID Google Spreadsheet',
    description='ID Sheet S1:',
    style={'description_width': 'initial'},
    layout={'display': 'none'} # Nascosto di default, mostrato da aggiorna_visibilita
)
gsheet_tab_s1_input = widgets.Text(
    value='ETF_EIMI', # Il tuo nome foglio/tab di default per Serie 1
    placeholder='Inserisci Nome Foglio/Tab',
    description='Nome Foglio S1:',
    style={'description_width': 'initial'},
    layout={'display': 'none'} # Nascosto di default
)

gsheet_id_s2_input = widgets.Text(
    value='', # Lascia vuoto o metti un ID Spreadsheet di default per Serie 2
    placeholder='Inserisci ID Google Spreadsheet',
    description='ID Sheet S2:',
    style={'description_width': 'initial'},
    layout={'display': 'none'} # Nascosto di default
)
gsheet_tab_s2_input = widgets.Text(
    value='', # Lascia vuoto o metti un nome foglio/tab di default per Serie 2
    placeholder='Inserisci Nome Foglio/Tab',
    description='Nome Foglio S2:',
    style={'description_width': 'initial'},
    layout={'display': 'none'} # Nascosto di default
)
# ---> FINE BLOCCO WIDGET PER GOOGLE SHEETS <---

# Upload file
file1_upload = widgets.FileUpload(description='Carica File S1', accept='.csv,.xls,.xlsx', multiple=False)
file2_upload = widgets.FileUpload(description='Carica File S2', accept='.csv,.xls,.xlsx', multiple=False)

# Data picker
start_date_picker = widgets.DatePicker(description='Data inizio:', value=datetime.date(2020, 1, 1))
end_date_picker = widgets.DatePicker(description='Data fine:', value=datetime.date.today())

# Impostazioni GARCH
scelta_distribuzione = widgets.Dropdown(
    options=['normal', 't', 'skewt', 'ged'], value='t', description='Distribuzione GARCH:', style={'description_width': 'initial'}
)
scelta_giorni_forecast = widgets.IntSlider(
    value=30, min=1, max=252, step=1, description='Orizzonte Forecast (giorni):', style={'description_width': 'initial'}, readout_format='d'
)

# ---> AGGIUNGI QUESTO NUOVO WIDGET QUI <---
# NUOVO WIDGET per il numero di simulazioni Monte Carlo
num_sim_mc_widget = widgets.IntSlider(
    value=2000, # Valore di default che preferisci (es. 2000)
    min=500,
    max=10000,
    step=500,
    description='Num. Sim. MC:',
    style={'description_width': 'initial'},
    readout_format='d'
)
# ---> FINE AGGIUNTA NUOVO WIDGET <---

# Opzioni Esportazione
formato_export = widgets.Dropdown(
    options=['CSV', 'Excel'], value='CSV', description='Formato Esportazione:', style={'description_width': 'initial'}
)
nome_file_export = widgets.Text(
    value='risultati_analisi', description='Nome Base File Esportazione:', style={'description_width': 'initial'}
)

# Pulsanti di esecuzione
btn_avvia = widgets.Button(description='Esegui Analisi', button_style='success', icon='cogs', layout=widgets.Layout(width='auto'))
btn_montecarlo = widgets.Button(description='Esegui Monte Carlo', button_style='info', icon='dice', tooltip='Esegue Monte Carlo sul modello fittato (priorità a Serie 1)', layout=widgets.Layout(width='auto'))

# Output areas
output_principale = widgets.Output() # Per log analisi principale, GARCH, etc.
output_sim_mc = widgets.Output() # Per output specifico Monte Carlo (es. Fan Chart)
#display(output_sim_mc)
# SOSTITUISCI la riga `display(output_sim_mc)` con questo blocco finale

# --- Aree di Output ---
# Definisci i contenitori
output_principale = widgets.Output() # Per log e grafici dell'analisi GARCH
output_sim_mc = widgets.Output()     # Per i risultati specifici del Monte Carlo

# Ora visualizza le aree con delle etichette per chiarezza
display(HTML("<h3>Log e Risultati Analisi GARCH</h3>"))
display(output_principale) # <-- QUESTA RIGA MANCANTE RISOLVE IL PROBLEMA

display(HTML("<hr><h3>Risultati Simulazione Monte Carlo</h3>"))
display(output_sim_mc)

Output()

Output()

## 7. Logica di Visibilità Widget

In [ ]:
def aggiorna_visibilita(change=None): # Il parametro 'change' è per l'observer, può essere omesso se chiami manualmente
    # Visibilità generale per i controlli della Serie 2
    is_confronto = tipo_analisi.value == 'Confronto'

    # Lista dei widget della Serie 2 la cui visibilità dipende da 'is_confronto'
    widgets_serie2_base = [fonte2_dropdown]
    # Includi gli altri widget della Serie 2 solo se sono stati definiti
    if 'ticker2_input' in globals(): widgets_serie2_base.append(ticker2_input)
    if 'gsheet_id_s2_input' in globals(): widgets_serie2_base.append(gsheet_id_s2_input)
    if 'gsheet_tab_s2_input' in globals(): widgets_serie2_base.append(gsheet_tab_s2_input)
    if 'file2_upload' in globals(): widgets_serie2_base.append(file2_upload)

    for w_s2 in widgets_serie2_base:
        w_s2.layout.display = 'flex' if is_confronto else 'none' # Usa 'flex' per VBox/HBox items

    # Visibilità specifica per fonte - SERIE 1
    fonte_s1_selezionata = fonte1_dropdown.value

    # Ticker/ID API per Serie 1 è visibile solo se la fonte NON è Google Sheets e NON è File Excel/CSV
    ticker1_input.layout.display = 'flex' if fonte_s1_selezionata not in ['Google Sheets', 'File Excel/CSV'] else 'none'

    # Widget Google Sheets per Serie 1
    gsheet_id_s1_input.layout.display = 'flex' if fonte_s1_selezionata == 'Google Sheets' else 'none'
    gsheet_tab_s1_input.layout.display = 'flex' if fonte_s1_selezionata == 'Google Sheets' else 'none'

    # File Upload per Serie 1
    file1_upload.layout.display = 'flex' if fonte_s1_selezionata == 'File Excel/CSV' else 'none'

    # Aggiorna placeholder/description per ticker1_input se visibile
    if ticker1_input.layout.display == 'flex':
        ticker1_input.description = 'Ticker/ID API S1:'
        ticker1_input.placeholder = 'Es. AAPL per API'


    # Visibilità specifica per fonte - SERIE 2 (solo se la modalità Confronto è attiva)
    if is_confronto:
        fonte_s2_selezionata = fonte2_dropdown.value
        ticker2_input.layout.display = 'flex' if fonte_s2_selezionata not in ['Google Sheets', 'File Excel/CSV'] else 'none'
        gsheet_id_s2_input.layout.display = 'flex' if fonte_s2_selezionata == 'Google Sheets' else 'none'
        gsheet_tab_s2_input.layout.display = 'flex' if fonte_s2_selezionata == 'Google Sheets' else 'none'
        file2_upload.layout.display = 'flex' if fonte_s2_selezionata == 'File Excel/CSV' else 'none'

        if ticker2_input.layout.display == 'flex':
             ticker2_input.description = 'Ticker/ID API S2:'
             ticker2_input.placeholder = 'Es. MSFT per API'
    else:
        # Se non è in modalità confronto, assicurati che tutti i widget specifici della Serie 2 (oltre a fonte2_dropdown) siano nascosti
        # Questo viene già gestito dal loop iniziale su widgets_serie2_base se non è is_confronto,
        # ma una doppia sicurezza per i campi dipendenti dalla fonte non fa male.
        for w_s2_specific in [ticker2_input, gsheet_id_s2_input, gsheet_tab_s2_input, file2_upload]:
             if hasattr(w_s2_specific, 'layout'): # Controlla se il widget è stato definito
                 w_s2_specific.layout.display = 'none'


# Collega la funzione agli eventi 'observe' dei dropdown
# Questi collegamenti attivano 'aggiorna_visibilita' ogni volta che il valore di questi widget cambia.
tipo_analisi.observe(aggiorna_visibilita, names='value')
fonte1_dropdown.observe(aggiorna_visibilita, names='value')
fonte2_dropdown.observe(aggiorna_visibilita, names='value')

# La chiamata iniziale aggiorna_visibilita(None) per impostare lo stato
# all'avvio del notebook va fatta ALLA FINE della Cella 22, dopo che
# `display(interfaccia_completa_layout)` è stata chiamata.
# Non metterla qui, altrimenti i widget potrebbero non essere ancora stati aggiunti al layout completo.



## 8. Funzione Principale di Analisi (`esegui_analisi`)

In [ ]:
# ===== FUNZIONI CHIAVE DI ANALISI =====
# CELLA 18 (Consolidata e CORRETTA)

from typing import Dict, Any
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from arch import arch_model
from scipy.stats import norm, t, chi2, gennorm # Aggiunto gennorm
from statsmodels.stats.diagnostic import acorr_ljungbox
import scipy.stats as stats # Per stats.probplot e stats.gennorm
import traceback
from arch.univariate.distribution import SkewStudent # Per il calcolo del quantile SkewT
import plotly.graph_objects as go
# from plotly.subplots import make_subplots # Potrebbe servire per grafici più complessi

# --- FUNZIONI HELPER PER IL CALCOLO DEL QUANTILE ---

def _safe_ppf(dist_obj, alpha: float, params_series):
    """
    Restituisce il quantile `alpha` per la distribuzione di `arch`
    indicata da `dist_obj` tentando prima la chiamata nativa, poi – se
    serve – ricostruendo la lista dei parametri di forma nel giusto
    ordine.

    Parameters
    ----------
    dist_obj : arch.univariate.base.Distribution
        Oggetto distribuzione del modello (es. Normal, Student, SkewStudent…).
    alpha : float
        Livello di probabilità (es. 0.05 per il 5%).
    params_series : pandas.Series | dict
        Serie/dizionario con i parametri stimati del modello
        (`fit_res.params`).

    Returns
    -------
    float
        Quantile della distribuzione (es. -1.645 per Normale al 5%).
    """
    q = [alpha]        # arch vuole sempre una lista/array per q

    # 1) tentativo diretto: funziona se la distribuzione “sa” già i suoi parametri
    try:
        return dist_obj.ppf(q)[0]
    except Exception:
        pass  # proseguiamo con il fallback intelligente

    # 2) ricostruisco l’elenco dei parametri di forma nell’ordine corretto
    try:
        shape_names = getattr(dist_obj, "parameter_names", lambda: [])()
        shape_params = [params_series[name] for name in shape_names]
        # arch vuole TUTTI i shape-parameters come *unica* lista in 2ª posizione
        return dist_obj.ppf(q, shape_params)[0]
    except Exception as e:
        raise RuntimeError(
            f"ppf fallita per {dist_obj.name} anche con shape_params: {e}"
        ) from e


def _fallback_quantile(dist_name: str, alpha: float, params_series):
    """
    Quantile di ripiego con scipy.stats se _safe_ppf fallisce.
    """
    d = dist_name.lower()

    # Skew-t → approssimo con t simmetrica df = eta
    if "skew" in d:
        eta = params_series.get("eta", 8.0)
        return t.ppf(alpha, df=eta)

    # Student's t
    if "t" in d:
        nu = params_series.get("nu", 8.0)
        return t.ppf(alpha, df=nu)

    # GED → scipy generalized normal
    if "ged" in d:
        nu = params_series.get("nu", 2.0)
        try:
            return gennorm.ppf(alpha, beta=nu)
        except Exception:
            return norm.ppf(alpha)

    # default → Normale standard
    return norm.ppf(alpha)

def calcola_quantile_var(modello_garch_fittato, livello_alpha=0.05):
    dist_obj = modello_garch_fittato.model.distribution
    params_stimati = modello_garch_fittato.params
    mu_punti_pct = params_stimati.get('mu', 0.0)
    quantile_calcolato = np.nan

    print(f"\n--- Calcolo Quantile/VaR 1g per '{dist_obj.name}' (alpha={livello_alpha*100:.0f}%) ---")
    try:
        quantile_calcolato = _safe_ppf(dist_obj, livello_alpha, params_stimati)
    except RuntimeError as e_safe_ppf:
        print(f"    AVVISO: _safe_ppf fallita ({e_safe_ppf}). Chiamo _fallback_quantile...")
        quantile_calcolato = _fallback_quantile(dist_obj.name, livello_alpha, params_stimati)

    if np.isnan(quantile_calcolato):
        print(f"    AVVISO DEFINITIVO: Quantile 1g ancora NaN. Uso Normale standard.")
        quantile_calcolato = norm.ppf(livello_alpha)

    try:
        sigma_forecast_punti_pct = float(np.sqrt(modello_garch_fittato.forecast(horizon=1).variance.iloc[-1, 0]))
    except Exception as e_sigma:
        print(f"❌ Errore nel calcolare sigma_forecast per VaR 1g: {e_sigma}. Imposto a NaN.")
        return np.nan, np.nan, np.nan

    var_1d_punti_pct = mu_punti_pct + quantile_calcolato * sigma_forecast_punti_pct

    print(f"    Quantile {livello_alpha*100:.0f}% (finale usato): {quantile_calcolato:.4f}")
    print(f"🔍 Forecast σ (1g, %): {sigma_forecast_punti_pct:.4f}%")
    print(f"📉 VaR (1g, {livello_alpha*100:.0f}%, Dist: '{dist_obj.name.lower()}'): {var_1d_punti_pct:.4f}%")

    return var_1d_punti_pct, quantile_calcolato, sigma_forecast_punti_pct
# --- FINE FUNZIONI HELPER E calcola_quantile_var ---

from statsmodels.tsa.stattools import acf

def interpreta_autocorrelazione(series, nome_serie: str, alpha=0.05):
    """
    Calcola l'ACF di una serie e restituisce un'interpretazione testuale,
    controllando se i valori superano i limiti di confidenza.
    """
    try:
        # Calcola i valori di autocorrelazione e gli intervalli di confidenza
        acf_values, confint = acf(series.dropna(), nlags=10, alpha=alpha)

        # Controlla i lag da 1 in poi (il lag 0 è sempre 1)
        for lag in range(1, len(acf_values)):
            # Se il valore di autocorrelazione è fuori dall'intervallo di confidenza...
            if abs(acf_values[lag]) > (confint[lag, 1] - acf_values[lag]):
                return f"⚠️  Potenziale autocorrelazione significativa rilevata per '{nome_serie}'."

        # Se nessun lag significativo è stato trovato
        return f"✅  Nessuna autocorrelazione significativa rilevata per '{nome_serie}'."

    except Exception as e:
        return f"❌ Errore durante il calcolo dell'ACF per '{nome_serie}': {e}"


def simulazione_montecarlo_multiperiodo(
    results,
    horizon: int = 30,
    n_sim: int = 2000,
    alpha_decimal: float = 0.05,
    starting_price: float = 100.0
):
    # ... (IL TUO CODICE ESISTENTE E CORRETTO PER QUESTA FUNZIONE VA QUI) ...
    # ... (L'indentazione di questa funzione nel tuo codice precedente sembrava corretta)
    print(f"\n🚀 Avvio Simulazione Monte Carlo Multiperiodo per {horizon} giorni con {n_sim} percorsi...")
    print(f"    Modello stimato: {type(results.model).__name__}")
    print(f"    Distribuzione:    {getattr(results.model.distribution, 'name', 'N/A')}")
    try:
        sim_forecast_values = results.forecast(horizon=horizon, method="simulation", simulations=n_sim).simulations.values
        sim_returns_daily_points_pct = None
        if sim_forecast_values.ndim == 3: sim_returns_daily_points_pct = sim_forecast_values[0].T
        elif sim_forecast_values.ndim == 2: sim_returns_daily_points_pct = sim_forecast_values
        else: raise ValueError(f"Shape inattesa: {sim_forecast_values.shape}")
        print(f"    Shape rendimenti giornalieri (punti %): {sim_returns_daily_points_pct.shape}")
        sim_returns_daily_decimal_actual = sim_returns_daily_points_pct / 100.0
        rendimenti_aggregati_decimal = sim_returns_daily_decimal_actual.sum(axis=0)
        print(f"    Rendimenti aggregati decimali su {horizon} giorni calcolati.")
        var_mc_agg_ret_decimal = np.percentile(rendimenti_aggregati_decimal, alpha_decimal * 100)
        cvar_mc_agg_ret_decimal = rendimenti_aggregati_decimal[rendimenti_aggregati_decimal <= var_mc_agg_ret_decimal].mean()
        simulated_prices = starting_price * (1 + sim_returns_daily_decimal_actual).cumprod(axis=0)
        print(f"    VaR ({alpha_decimal*100:.0f}%) MC rend. aggregato (dec): {var_mc_agg_ret_decimal:.6f} ({var_mc_agg_ret_decimal*100:.2f}%)")
        print(f"    CVaR ({alpha_decimal*100:.0f}%) MC rend. aggregato (dec): {cvar_mc_agg_ret_decimal:.6f} ({cvar_mc_agg_ret_decimal*100:.2f}%)")
        print("✅ Simulazione Monte Carlo Multiperiodo completata.")
        return sim_returns_daily_decimal_actual, var_mc_agg_ret_decimal, cvar_mc_agg_ret_decimal, simulated_prices
    except Exception as e:
        print(f"❌ Errore durante la simulazione Monte Carlo multiperiodo: {e}")
        traceback.print_exc()
        return np.array([]), np.nan, np.nan, np.array([])

def _esegui_analisi_garch_dettagliata(
    dati_dict: Dict[str, pd.DataFrame],
    selected_dist: str,
    selected_model: str,
    num_simulazioni_mc: int,
    horizon_mc: int,
    alpha_mc_decimal: float,
    out_area=None # Non più utilizzato
) -> Dict[str, Any]:

    results_output_dict = {}

    def _map_dist(dist_name: str) -> str:
        d = dist_name.lower()
        if d in {"t", "t-student", "studentst"}: return "t"
        if d in {"skewt", "skewstudent"}: return "skewt"
        if d == "ged": return "ged"
        return "normal"
    dist_for_arch = _map_dist(selected_dist)

    for label_serie, df_input in dati_dict.items():
        results_output_dict[label_serie] = {}
        print(f"\n📊 Analisi per: {label_serie}")

        rendimenti_decimali = df_input["returns"].dropna()
        if rendimenti_decimali.empty:
            results_output_dict[label_serie]['errore'] = "Serie rendimenti vuota"
            continue

        # --- 1. PREPARA GRAFICO RENDIMENTI ---
        fig_rend = go.Figure()
        fig_rend.add_trace(go.Scatter(x=rendimenti_decimali.index, y=rendimenti_decimali * 100, mode="lines", name="Rendimenti (%)"))
        fig_rend.update_layout(title_text=f"Rendimenti Giornalieri (%) – {label_serie}")
        results_output_dict[label_serie]['fig_rendimenti'] = fig_rend

        # In Cella 18, dopo la creazione di 'fig_rend'

        # --- 1b. PREPARA GRAFICI DIAGNOSTICI PRE-FIT ---
        print("   1b. Preparazione grafici ACF/PACF e KDE...")

        # Grafico ACF/PACF
        fig_acf = mostra_grafici_acf_pacf(returns_series=rendimenti_decimali, label_serie=label_serie)
        if fig_acf:
           results_output_dict[label_serie]['fig_acf_pacf'] = fig_acf
           # --- BLOCCO DI INTERPRETAZIONE TESTUALE ACF ---
        print("\n   Interpretazione Testuale Autocorrelazione:")

        # Controlla i rendimenti semplici
        risultato_acf_returns = interpreta_autocorrelazione(rendimenti_decimali, "Rendimenti Semplici")
        print(f"   - {risultato_acf_returns}")

        # Controlla i rendimenti al quadrato (per effetti ARCH)
        risultato_acf_sq_returns = interpreta_autocorrelazione(rendimenti_decimali**2, "Rendimenti al Quadrato")
        print(f"   - {risultato_acf_sq_returns}")

        # Grafico di Densità (KDE)

        var_storico_dec, cvar_storico_dec = calcola_var_cvar_storico(rendimenti_decimali, alpha_mc_decimal)
        fig_kde = mostra_kde_var_cvar(
                rendimenti_decimali,
                var_storico_dec,
                cvar_storico_dec,
                title=f"Distribuzione Storica Rendimenti per {label_serie}"
        )
        if fig_kde:
                results_output_dict[label_serie]['fig_kde'] = fig_kde





        # --- 2. FIT DEL MODELLO GARCH ---
        print("   2. Fit del modello GARCH in corso (modalità automatica)...")

        # Usiamo sempre la griglia completa e robusta per trovare il miglior modello
        base_grid = [
            ("GARCH",  1, 0, 1),  # GARCH(1,1)
            ("GARCH",  2, 0, 2),  # GARCH(2,2)
            ("GARCH",  1, 1, 1),  # GJR-GARCH(1,1) -> per effetti di leva
            ("GARCH",  2, 1, 2),  # GJR-GARCH(2,2)
            ("EGARCH", 1, 1, 1),  # EGARCH(1,1)
            ("EGARCH", 2, 1, 2)   # EGARCH(2,2)
        ]

        print("   Griglia modelli da provare:")
        for spec in base_grid:
            print(f"      {spec}")

        best_aic = np.inf
        best_fit_model = None
        best_model_spec = None

        # Il loop di stima che segue è corretto e rimane invariato
        for vol_type, p, o, q in base_grid:
            try:
                am = arch_model(rendimenti_decimali * 100, mean="ARX", lags=[1, 2], vol=vol_type, p=p, o=o, q=q, dist=dist_for_arch, rescale=False)
                fit = am.fit(disp="off", show_warning=False)
                if fit.aic < best_aic:
                    best_aic, best_fit_model, best_model_spec = fit.aic, fit, (vol_type, p, o, q)
            except Exception as e:
                print(f"⚠️ Errore durante il fit del modello {vol_type}({p},{o},{q}): {e}")
                continue

        if best_fit_model is None:
            results_output_dict[label_serie]['errore'] = "Fit del modello GARCH fallito."
            print(f"❌ Fit GARCH fallito per {label_serie}.")
            # Aggiunto 'continue' per passare alla prossima serie se il fit fallisce
            continue

        best_model_name = f"{best_model_spec[0]}({best_model_spec[1]},{best_model_spec[2]},{best_model_spec[3]})"
        print(f"   ✅ Modello selezionato: {best_model_name} (AIC: {best_aic:.2f})")
        print(best_fit_model.summary())





        # --- 3. DIAGNOSTICA, TEST STATISTICI E BACKTESTING ---
        std_resid = best_fit_model.std_resid.dropna()
        if not std_resid.empty:
            print("\n   3. Esecuzione diagnostica e backtesting...")


            # PREPARA Q-Q PLOT
            dist_params=(); dist_obj=None; dist_label="N/A"
            dist_name_lower = best_fit_model.model.distribution.name.lower()
            if 'skew' in dist_name_lower:
                dist_obj, df_param = stats.t, best_fit_model.params.get('eta')
                if df_param is not None: dist_label, dist_params = f"t-Student (df={df_param:.2f})", (df_param,)
            elif 't' in dist_name_lower:
                dist_obj, df_param = stats.t, best_fit_model.params.get('nu')
                if df_param is not None: dist_label, dist_params = f"t-Student (df={df_param:.2f})", (df_param,)
            else: dist_obj, dist_label = stats.norm, "Normale"
            if dist_obj:
                osm, osr = stats.probplot(std_resid, dist=dist_obj, sparams=dist_params, fit=False)
                fig_qq = go.Figure()
                fig_qq.add_trace(go.Scatter(x=osm, y=osr, mode='markers', name='Quantili Osservati'))
                fig_qq.add_trace(go.Scatter(x=osm, y=osm, mode='lines', name='Linea Teorica', line=dict(color='red', dash='dash')))
                fig_qq.update_layout(title_text=f'Q-Q Plot Residui vs {dist_label} ({label_serie})')
                results_output_dict[label_serie]['fig_qq'] = fig_qq

            # --- CREA GRAFICO CONFRONTO VOLATILITÀ ---
            print("   Creazione grafico confronto volatilità...")
            fig_confronto_vol = mostra_confronto_volatilita(
                df_dati=df_input,
                fitted_model=best_fit_model,
                label_serie=label_serie
            )
            if fig_confronto_vol:
                results_output_dict[label_serie]['fig_confronto_vol'] = fig_confronto_vol

            # TEST LJUNG-BOX
            lags_lb = min(10, len(std_resid) // 5)
            if lags_lb > 0:
                lb_results_df = acorr_ljungbox(std_resid**2, lags=[lags_lb], return_df=True)
                print("\n   Test di Ljung-Box sui Residui Standardizzati al Quadrato:")
                print(lb_results_df)

                p_value = lb_results_df['lb_pvalue'].iloc[0]
                print(f"\n   Interpretazione del Test:")
                if p_value < 0.05:
                    print(f"   ⚠️ Risultato: P-value ({p_value:.4f}) è basso. Il test suggerisce che ci sono ancora effetti ARCH non catturati dal modello.")
                else:
                    print(f"   ✅ Risultato: P-value ({p_value:.4f}) è alto. Il modello sembra aver catturato bene la struttura della volatilità.")





            # CALCOLO VAR A 1 GIORNO
            var_1d_final_pct_pts, quantile_1g, _ = calcola_quantile_var(best_fit_model, alpha_mc_decimal)

            # BLOCCO BACKTESTING CON TEST Kupienc e Christoffersen
            rendimenti_per_backtest_pct_pts_bk = rendimenti_decimali * 100
            idx_comune_bk = rendimenti_per_backtest_pct_pts_bk.index.intersection(best_fit_model.conditional_volatility.index)
            if not idx_comune_bk.empty and not np.isnan(quantile_1g):
                cond_vol_pct_pts_bk_final = best_fit_model.conditional_volatility.loc[idx_comune_bk]
                aligned_returns_pct_pts_bk_final = rendimenti_per_backtest_pct_pts_bk.loc[idx_comune_bk]
                VaR_series_pct_pts_bk = cond_vol_pct_pts_bk_final * quantile_1g
                hits_bk = (aligned_returns_pct_pts_bk_final < VaR_series_pct_pts_bk).astype(int)

                # Test di Kupiec
                n_viol, n_obs, p_exp = hits_bk.sum(), len(hits_bk), alpha_mc_decimal
                print(f"\n   --- 🧪 Backtesting del VaR ---")
                print(f"     Test di Kupiec (POF): Oss.={n_obs}, Viol.={n_viol}, Tasso Oss.={n_viol/n_obs:.4f} (Atteso={p_exp:.4f})")
                logL0k, logL1k = 0, 0
                if p_exp > 0 and p_exp < 1: logL0k = (n_obs - n_viol)*np.log(1-p_exp) + (n_viol*np.log(p_exp) if n_viol > 0 else 0)
                viol_rate_k = n_viol / n_obs
                if viol_rate_k > 0 and viol_rate_k < 1: logL1k = (n_obs - n_viol)*np.log(1-viol_rate_k) + (n_viol*np.log(viol_rate_k) if n_viol > 0 else 0)
                LR_POF_bk = -2 * (logL0k - logL1k)
                pv_k_bk = 1 - chi2.cdf(LR_POF_bk, df=1)
                print(f"     LR_POF: {LR_POF_bk:.4f}, P-val: {pv_k_bk:.4f} - {'✅ ACCETTATO' if pv_k_bk >= 0.05 else '❌ RIFIUTATO'}")

                # Test di Christoffersen e Congiunto
                if len(hits_bk) >= 2:
                    h_shift, h_curr = hits_bk.shift(1).dropna(), hits_bk.loc[hits_bk.shift(1).dropna().index]
                    T00, T01, T10, T11 = ((h_shift==0)&(h_curr==0)).sum(), ((h_shift==0)&(h_curr==1)).sum(), ((h_shift==1)&(h_curr==0)).sum(), ((h_shift==1)&(h_curr==1)).sum()
                    pi01 = T01/(T00+T01) if (T00+T01)>0 else 0; pi11 = T11/(T10+T11) if (T10+T11)>0 else 0
                    pi_hat = (T01+T11)/(T00+T01+T10+T11) if (T00+T01+T10+T11)>0 else 0
                    logL0i, logL1i = 0,0
                    if pi_hat>0 and pi_hat<1: logL0i = (T00+T10)*np.log(1-pi_hat) + (T01+T11)*np.log(pi_hat)
                    termsL1i = []
                    if T00>0 and (1-pi01)>0: termsL1i.append(T00*np.log(1-pi01))
                    if T01>0 and pi01>0: termsL1i.append(T01*np.log(pi01))
                    if T10>0 and (1-pi11)>0: termsL1i.append(T10*np.log(1-pi11))
                    if T11>0 and pi11>0: termsL1i.append(T11*np.log(pi11))
                    logL1i = sum(termsL1i) if termsL1i else 0.0
                    LR_IND_bk = -2 * (logL0i - logL1i)
                    pv_i_bk = 1 - chi2.cdf(LR_IND_bk, df=1)
                    print(f"     Test di Christoffersen (Indipendenza): LR_IND: {LR_IND_bk:.4f}, P-val: {pv_i_bk:.4f} - {'✅ INDIPENDENTI' if pv_i_bk >= 0.05 else '❌ RIFIUTATA'}")
                    LR_CC_bk = LR_POF_bk + LR_IND_bk
                    pv_cc_bk = 1 - chi2.cdf(LR_CC_bk, df=2)
                    print(f"     Test Congiunto: LR_CC: {LR_CC_bk:.4f}, P-val: {pv_cc_bk:.4f} - {'✅ ACCETTATO' if pv_cc_bk >= 0.05 else '❌ RIFIUTATO'}")

                # PREPARA GRAFICO BACKTESTING
                fig_backtest_var = go.Figure()
                fig_backtest_var.add_trace(go.Scatter(x=aligned_returns_pct_pts_bk_final.index, y=aligned_returns_pct_pts_bk_final, mode='lines', name='Rendimenti (%)', line=dict(color='grey', width=1)))
                fig_backtest_var.add_trace(go.Scatter(x=VaR_series_pct_pts_bk.index, y=VaR_series_pct_pts_bk, mode='lines', name=f'VaR {alpha_mc_decimal*100:.0f}% Previsto', line=dict(color='blue', dash='dash')))
                if hits_bk.sum() > 0:
                    fig_backtest_var.add_trace(go.Scatter(x=aligned_returns_pct_pts_bk_final.index[hits_bk == 1], y=aligned_returns_pct_pts_bk_final[hits_bk == 1], mode='markers', name='Violazioni VaR', marker=dict(color='red', size=8)))
                fig_backtest_var.update_layout(title=f'Backtesting VaR {alpha_mc_decimal*100:.0f}% per {label_serie}')
                results_output_dict[label_serie]['fig_backtest_var'] = fig_backtest_var

        # --- 5. ESECUZIONE E PREPARAZIONE GRAFICI MONTECARLO ---
        print("\n   4. Esecuzione e preparazione grafici Monte Carlo...")
        last_close_price = df_input["Close"].dropna().iloc[-1]
        sim_ret_dec, var_mc, cvar_mc, prezzi_sim = simulazione_montecarlo_multiperiodo(
            results=best_fit_model, horizon=horizon_mc, n_sim=num_simulazioni_mc,
            alpha_decimal=alpha_mc_decimal, starting_price=last_close_price
        )
        if sim_ret_dec is not None and sim_ret_dec.size > 0:
            fig_fan_returns = fan_chart_plotly(sim_ret_dec, titolo=f"Fan-chart Rendimenti MC (%) - {label_serie}")
            results_output_dict[label_serie]['fig_mc_returns'] = fig_fan_returns
        if prezzi_sim is not None and prezzi_sim.size > 0:
            fig_prezzi = plot_prezzi_simulati_plotly(prezzi_sim, titolo=f"Andamento Prezzi Simulati MC - {label_serie}", start_date=df_input.index[-1], historical_prices=df_input['Close'].dropna().iloc[-60:])
            results_output_dict[label_serie]['fig_mc_prices'] = fig_prezzi

        # --- 6. AGGIUNTA RISULTATI FINALI AL DIZIONARIO ---
        results_output_dict[label_serie].update({
            "risultato_completo_garch_fit": best_fit_model, "modello_tipo": best_model_name,
            "aic": best_aic, "VaR_mc_orizzonte_agg_ret_dec": var_mc,
            "CVaR_mc_orizzonte_agg_ret_dec": cvar_mc,
        })
        print(f"--- ✅ Analisi per {label_serie} completata ---")

    return results_output_dict

## 9. Funzione Callback per Simulazione Monte Carlo (`on_btn_montecarlo_clicked`)

In [ ]:
# ===== FUNZIONE PER IL PULSANTE MONTE CARLO PREZZI =====
# CELLA 20 (ID: 8588c36b) - (MODIFICATA E CORRETTA)
"""
from IPython.display import display, HTML, clear_output
import pandas as pd # Aggiunto per isinstance
# Assicurati che i widget (btn_avvia, tipo_analisi, num_sim_mc_widget) e
# le funzioni (simulazione_montecarlo_multiperiodo, fan_chart_plotly, plot_prezzi_simulati_plotly)
# siano definite nelle celle precedenti.

def on_btn_montecarlo_clicked(b_mc):
    with output_sim_mc:
        clear_output(wait=True)
        display(HTML("<h4>🎲 Avvio Simulazione Monte Carlo Prezzi...</h4>"))

        dati_storici_dict = getattr(btn_avvia, "dati_per_analisi", {})
        tipo_analisi_selezionato = tipo_analisi.value

        # Leggi il numero di simulazioni dal widget (deve essere definito nella Cella 14)
        n_sim_mc_globale = num_sim_mc_widget.value if 'num_sim_mc_widget' in globals() else 2000

        serie_da_simulare = []
        if "Serie 1" in dati_storici_dict and getattr(btn_avvia, "fitted_model1", None) is not None:
            serie_da_simulare.append({
                "label": "Serie 1",
                "model": getattr(btn_avvia, "fitted_model1"),
                "df_storico": dati_storici_dict["Serie 1"]
            })

        if tipo_analisi_selezionato == "Confronto":
            if "Serie 2" in dati_storici_dict and getattr(btn_avvia, "fitted_model2", None) is not None:
                serie_da_simulare.append({
                    "label": "Serie 2",
                    "model": getattr(btn_avvia, "fitted_model2"),
                    "df_storico": dati_storici_dict["Serie 2"]
                })
            elif "Serie 2" not in dati_storici_dict or getattr(btn_avvia, "fitted_model2", None) is None:
                print("AVVISO: Modalità confronto selezionata, ma dati o modello per Serie 2 non disponibili. Eseguo solo per Serie 1 (se disponibile).")

        if not serie_da_simulare:
            print("❌ Nessun modello GARCH valido o dati storici disponibili per la simulazione.")
            print("   Esegui prima l’analisi principale.")
            return

        for dati_serie in serie_da_simulare:
            label_sim = dati_serie["label"]
            model_to_use = dati_serie["model"]
            df_prezzi_storici = dati_serie["df_storico"]

            display(HTML(f"<h5>--- Simulazione Monte Carlo per {label_sim} ---</h5>"))

            if df_prezzi_storici is None or "Close" not in df_prezzi_storici.columns or df_prezzi_storici["Close"].dropna().empty:
                print(f"❌ Dati 'Close' storici non disponibili o vuoti per {label_sim}. Salto simulazione per questa serie.")
                continue

            # DEBUG per il prezzo di partenza (opzionale)
            print("\n--- DEBUG: Ultime quotazioni 'Close' disponibili ---")
            print(df_prezzi_storici["Close"].dropna().tail())
            print("--------------------------------------------------\n")

            start_price_sim = df_prezzi_storici["Close"].dropna().iloc[-1]
            last_historical_date = df_prezzi_storici.index[-1] if isinstance(df_prezzi_storici.index, pd.DatetimeIndex) else None

            print(f"📌 Prezzo di partenza simulazione ({label_sim}): {start_price_sim:.4f}")
            if last_historical_date:
                print(f"🗓️ Ultima data storica usata come riferimento: {last_historical_date.strftime('%Y-%m-%d')}")

            horizon_sim = scelta_giorni_forecast.value
            alpha_mc_sim_dec = 0.05

            sim_ret_dec, var_agg_ret_dec, cvar_agg_ret_dec, prezzi_simulati_plot = simulazione_montecarlo_multiperiodo(
                results=model_to_use,
                horizon=horizon_sim,
                n_sim=n_sim_mc_globale, # <<< USA IL VALORE UNIFORME DAL WIDGET
                alpha_decimal=alpha_mc_sim_dec,
                starting_price=start_price_sim
            )

            if sim_ret_dec is None or sim_ret_dec.size == 0:
                print(f"❌ La simulazione Monte Carlo per {label_sim} non ha prodotto rendimenti validi.")
                continue

            # 1. Fan chart dei RENDIMENTI GIORNALIERI (%)
            fig_fan_returns = fan_chart_plotly(
                sim_ret_dec,
                titolo=f"Fan-chart Rendimenti Giornalieri MC (%) ({label_sim}) – {horizon_sim} gg",
                livello_var_decimal=alpha_mc_sim_dec
            )
            if fig_fan_returns:
                # ---> CORREZIONE: Mostra il grafico Plotly con fig.show() <---
                show_plotly(fig_fan_returns, clear=False)

            # 2. Plot dei PREZZI SIMULATI (che ora è Plotly)
            if prezzi_simulati_plot is not None and prezzi_simulati_plot.size > 0:
                historical_prices_for_plot = None
                if 'Close' in df_prezzi_storici:
                    historical_prices_for_plot = df_prezzi_storici['Close'].dropna().iloc[-60:]

                # Usa la nuova funzione plot_prezzi_simulati_plotly (da definire nella Cella 12)
                fig_prezzi = plot_prezzi_simulati_plotly(
                    prezzi_simulati_plot,
                    titolo=f"Andamento Prezzi Simulati ({label_sim}) – Orizzonte {horizon_sim} gg",
                    start_date=last_historical_date,
                    historical_prices=historical_prices_for_plot
                )
                if fig_prezzi:
                    # ---> CORREZIONE: Mostra il grafico Plotly con fig.show() <---
                    show_plotly(fig_prezzi, clear=False)
                else:
                    print(f"❌ Grafico prezzi non generato per {label_sim}.")
            else:
                print(f"❌ Nessun prezzo simulato da plottare per {label_sim}.")

            print(f"📈 VaR 5% rend. aggregato su {horizon_sim}gg ({label_sim}): {var_agg_ret_dec*100:.2f}%")
            print(f"📈 CVaR 5% rend. aggregato su {horizon_sim}gg ({label_sim}): {cvar_agg_ret_dec*100:.2f}%")
            print("-" * 50)

    display(HTML("<p style='color:green;'>✅ Tutte le Simulazioni Monte Carlo Prezzi richieste sono state completate.</p>"))  """

'\nfrom IPython.display import display, HTML, clear_output\nimport pandas as pd # Aggiunto per isinstance\n# Assicurati che i widget (btn_avvia, tipo_analisi, num_sim_mc_widget) e\n# le funzioni (simulazione_montecarlo_multiperiodo, fan_chart_plotly, plot_prezzi_simulati_plotly)\n# siano definite nelle celle precedenti.\n\ndef on_btn_montecarlo_clicked(b_mc):\n    with output_sim_mc:\n        clear_output(wait=True)\n        display(HTML("<h4>🎲 Avvio Simulazione Monte Carlo Prezzi...</h4>"))\n\n        dati_storici_dict = getattr(btn_avvia, "dati_per_analisi", {})\n        tipo_analisi_selezionato = tipo_analisi.value\n\n        # Leggi il numero di simulazioni dal widget (deve essere definito nella Cella 14)\n        n_sim_mc_globale = num_sim_mc_widget.value if \'num_sim_mc_widget\' in globals() else 2000\n\n        serie_da_simulare = []\n        if "Serie 1" in dati_storici_dict and getattr(btn_avvia, "fitted_model1", None) is not None:\n            serie_da_simulare.append({\n    

## 10. Assemblaggio Interfaccia Utente e Avvio

In [ ]:
# SOSTITUISCI CON QUESTA NUOVA E SEMPLICE FUNZIONE
from IPython.display import clear_output

def show_plotly(fig, clear=False):
    # Se clear=True, pulisce l'output dell'intera cella.
    if clear:
        clear_output(wait=True)

    # Chiama direttamente fig.show(). Su Colab, con il renderer corretto,
    # questo disegnerà il grafico direttamente nell'output della cella.
    fig.show()

In [ ]:

# ===== CALCOLO METRICHE DI PERFORMANCE =====
import numpy as np

def calcola_metriche_performance(df_returns, risk_free_rate=0.0, periodi_annui=252):
    returns = df_returns['returns'].dropna()
    if returns.empty:
        return {}

    # Calcoli base
    mean_return = returns.mean()
    std_return = returns.std()
    downside_returns = returns[returns < 0]

    sharpe = np.sqrt(periodi_annui) * (mean_return - risk_free_rate) / std_return if std_return != 0 else np.nan
    sortino = np.sqrt(periodi_annui) * (mean_return - risk_free_rate) / downside_returns.std() if not downside_returns.empty else np.nan
    cumulative = (1 + returns).cumprod()
    max_drawdown = (cumulative / cumulative.cummax() - 1).min()
    calmar = (mean_return * periodi_annui) / abs(max_drawdown) if max_drawdown != 0 else np.nan

    return {
        'Sharpe Ratio': sharpe,
        'Sortino Ratio': sortino,
        'Calmar Ratio': calmar,
        'Max Drawdown': max_drawdown,
        'Annualized Return': mean_return * periodi_annui
    }




In [ ]:
# ===== CELLA 22 - VERSIONE FINALE E DEFINITIVA =====

import pandas as pd
import numpy as np
import io
import datetime
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

# Assicurati che le funzioni di utility (come ensure_decimal_returns, _load_data_from_...) e
# la funzione _esegui_analisi_garch_dettagliata (dalla Cella 18 CORRETTA) siano definite.
# Assicurati che tutti i widget di base (fonte1_dropdown, ecc.) siano definiti.
# ===== DEFINIZIONE DI TUTTI I WIDGET DELL'INTERFACCIA =====

style_descr_initial = {'description_width': 'initial'}
header_html = widgets.HTML("<h1>Tool di Analisi Finanziaria e Rischio</h1><p>Configura le serie storiche...</p><hr>")

# --- Widget di configurazione generale ---
tipo_analisi = widgets.Dropdown(options=["Analisi singola", "Confronto"], value="Analisi singola", description="Tipo di Analisi:", style=style_descr_initial)
start_date_picker = widgets.DatePicker(description='Data Inizio', value=datetime.date.today() - datetime.timedelta(days=365*3))
end_date_picker = widgets.DatePicker(description='Data Fine', value=datetime.date.today())
config_generale_box = widgets.VBox([widgets.HTML("<h2>Configurazione Generale</h2>"), tipo_analisi, widgets.HBox([start_date_picker, end_date_picker])], layout=widgets.Layout(margin='0 0 10px 0'))

# --- Widget per le fonti dati (Serie 1 e 2) ---
# (Assicurati che i widget come fonte1_dropdown, ticker1_input, ecc. siano definiti in una cella precedente)
box_serie1_widgets = widgets.VBox([widgets.HTML("<h3>Serie 1</h3>"), fonte1_dropdown, ticker1_input, gsheet_id_s1_input, gsheet_tab_s1_input, file1_upload], layout=widgets.Layout(padding="0 0 10px 0"))
box_serie2_widgets = widgets.VBox([widgets.HTML("<h3>Serie 2 (per Confronto)</h3>"), fonte2_dropdown, ticker2_input, gsheet_id_s2_input, gsheet_tab_s2_input, file2_upload], layout=widgets.Layout(padding="0 0 10px 0"))
box_data_sources = widgets.HBox([box_serie1_widgets, box_serie2_widgets], layout=widgets.Layout(justify_content='space-around'))

# --- Widget per le impostazioni del modello ---
# (Assicurati che scelta_distribuzione, scelta_giorni_forecast, num_sim_mc_widget, ecc. siano definiti prima)
scelta_alpha = widgets.Dropdown(options=[('95% (alpha=5%)', 0.05), ('99% (alpha=1%)', 0.01), ('97.5% (alpha=2.5%)', 0.025)], value=0.05, description='Livello Confidenza VaR:', style=style_descr_initial)
box_garch_settings = widgets.VBox([widgets.HTML("<h2>Impostazioni Modello e Output</h2>"), scelta_distribuzione, scelta_alpha, scelta_giorni_forecast, num_sim_mc_widget], layout=widgets.Layout(margin='10px 0'))
box_export_settings = widgets.VBox([formato_export, nome_file_export], layout=widgets.Layout(margin='10px 0'))
box_model_output_settings = widgets.HBox([box_garch_settings, box_export_settings], layout=widgets.Layout(justify_content='space-between'))

# --- Pulsanti e Aree di Output ---
# (Assicurati che btn_avvia, output_principale siano definiti)
box_buttons = widgets.HBox([btn_avvia], layout=widgets.Layout(justify_content='center', margin='20px 0'))
accordion_outputs = widgets.Accordion(children=[output_principale])
accordion_outputs.set_title(0, 'Risultati e Log Analisi')



def on_btn_esegui_tutto_clicked(b):
    # Pulisce l'output e mostra il messaggio di attesa
    with output_principale:
        clear_output(wait=True)
        print("⏳ Avvio analisi completa e simulazione, attendere prego...")
    accordion_outputs.selected_index = 0

    # --- 1. Logica di Caricamento Dati ---
    start_date_obj = start_date_picker.value
    end_date_obj = end_date_picker.value
    start_date_str = start_date_obj.strftime('%Y-%m-%d')
    end_date_str = end_date_obj.strftime('%Y-%m-%d')
    numero_simulazioni_globale = num_sim_mc_widget.value
    alpha_selezionato = scelta_alpha.value

    def get_data(fonte_widget, ticker_api_widget, gs_id_widget, gs_tab_widget, file_upload_widget_ref):
        fonte_selezionata = fonte_widget.value; df_loaded = None
        if fonte_selezionata in ['Alpha Vantage', 'FMP', 'Tiingo', 'EOD', 'Twelve Data']:
            ticker_val = ticker_api_widget.value; print(f"Caricamento dati da {fonte_selezionata} per '{ticker_val}'...")
            if not ticker_val: print(f"❌ Ticker non specificato."); return None
            api_key_to_use = {'Alpha Vantage': ALPHA_VANTAGE_API_KEY, 'FMP': FMP_API_KEY, 'Tiingo': TIINGO_API_KEY, 'EOD': EOD_API_KEY, 'Twelve Data': TWELVE_DATA_API_KEY}.get(fonte_selezionata)
            if not api_key_to_use: print(f"❌ Chiave API non configurata."); return None
            loader_func = {'Alpha Vantage': _load_data_from_alphavantage, 'FMP': _load_data_from_fmp,'Tiingo': _load_data_from_tiingo, 'EOD': _load_data_from_eod,'Twelve Data': _load_data_from_twelvedata,}.get(fonte_selezionata)
            if loader_func: df_loaded = loader_func(ticker_val, api_key_to_use, start_date_str, end_date_str)
        elif fonte_selezionata == "Google Sheets":
            sheet_id_val = gs_id_widget.value; worksheet_name_val = gs_tab_widget.value
            if not sheet_id_val or not worksheet_name_val: print("❌ ID o Nome Foglio non specificati."); return None
            df_loaded = _load_data_from_google_sheets(sheet_id_val, worksheet_name_val, SERVICE_ACCOUNT_FILE)
        elif fonte_selezionata == "File Excel/CSV":
            if file_upload_widget_ref and file_upload_widget_ref.value:
                file_info = list(file_upload_widget_ref.value.values())[0]; file_name = file_info['metadata']['name']
                file_content = file_info['content']; file_stream = io.BytesIO(file_content); file_stream.name = file_name
                df_loaded = carica_dati_da_file(file_stream)
        if df_loaded is not None and not df_loaded.empty:
            try:
                if not isinstance(df_loaded.index, pd.DatetimeIndex):
                    df_loaded.index = pd.to_datetime(df_loaded.index, errors='coerce'); df_loaded.dropna(subset=[df_loaded.index.name], inplace=True)
                start_ts = pd.to_datetime(start_date_str); end_ts = pd.to_datetime(end_date_str)
                df_loaded = df_loaded[(df_loaded.index >= start_ts) & (df_loaded.index <= end_ts.replace(hour=23, minute=59, second=59))]
                if df_loaded.empty: return None
                return df_loaded.copy()
            except Exception: return None
        return None

    dati_per_garch_engine = {}
    print("\n--- ⏳ Caricamento Dati Serie 1 ---")
    df1 = get_data(fonte1_dropdown, ticker1_input, gsheet_id_s1_input, gsheet_tab_s1_input, file1_upload)
    if df1 is not None and not df1.empty and 'Close' in df1.columns:
        df1['returns'] = ensure_decimal_returns(df1['Close'].pct_change())
        dati_per_garch_engine["Serie 1"] = df1.dropna(subset=['returns'])

    if tipo_analisi.value == "Confronto":
        print("\n--- ⏳ Caricamento Dati Serie 2 ---")
        df2 = get_data(fonte2_dropdown, ticker2_input, gsheet_id_s2_input, gsheet_tab_s2_input, file2_upload)
        if df2 is not None and not df2.empty and 'Close' in df2.columns:
            df2['returns'] = ensure_decimal_returns(df2['Close'].pct_change())
            dati_per_garch_engine["Serie 2"] = df2.dropna(subset=['returns'])

    if not dati_per_garch_engine:
        with output_principale: print("❌ Nessun dato valido caricato. Analisi interrotta."); return

    # --- 2. Chiamata alla funzione di analisi (Cella 18) che PREPARA tutto ---
    with output_principale:
        print(f"\n--- ⚙️ Esecuzione Analisi GARCH (Alpha={alpha_selezionato*100:.1f}%) ---")
        analysis_results = _esegui_analisi_garch_dettagliata(
            dati_dict=dati_per_garch_engine, selected_dist=scelta_distribuzione.value,
            selected_model="auto", num_simulazioni_mc=numero_simulazioni_globale,
            horizon_mc=scelta_giorni_forecast.value, alpha_mc_decimal=alpha_selezionato
        )

        # --- 3. Blocco di VISUALIZZAZIONE ORDINATA (CON INDENTAZIONE CORRETTA) ---
        print("\n--- 📊 Visualizzazione Risultati ---")
        print("\n--- DEBUG: ISPEZIONE DI 'analysis_results' ---")
    if not analysis_results:
        print("Il dizionario 'analysis_results' è VUOTO.")
    else:
        for label, results_dict in analysis_results.items():
            print(f"\n--- Contenuto per '{label}' ---")
            # Stampa tutte le chiavi (i nomi dei risultati) presenti nel dizionario
            print(f"Chiavi trovate: {list(results_dict.keys())}")
            print("--------------------------------------------------\n")


        for label, results in analysis_results.items():
            display(HTML(f"<h2>--- Risultati per {label} ---</h2>"))
            if "errore" in results:
                print(f"Errore: {results['errore']}")
                continue

            # Queste righe ora sono DENTRO il ciclo for
            if results.get('fig_rendimenti'):
                results['fig_rendimenti'].show()
            if results.get('fig_acf_pacf'):
                results['fig_acf_pacf'].show()
            if results.get('fig_kde'):
                results['fig_kde'].show()
            if results.get('fig_qq'):
                results['fig_qq'].show()
            if results.get('fig_confronto_vol'):
               results['fig_confronto_vol'].show()
            if results.get('fig_backtest_var'):
                results['fig_backtest_var'].show()

            display(HTML("<h3>Grafici Monte Carlo</h3>"))
            if results.get('fig_mc_returns'):
                results['fig_mc_returns'].show()
            if results.get('fig_mc_prices'):
                results['fig_mc_prices'].show()

        # --- Calcolo e stampa metriche di performance ---
        for label, df in dati_per_garch_engine.items():
            print(f"\n📊 Metriche di Performance per {label}")
            metriche = calcola_metriche_performance(df)
            for nome, valore in metriche.items():
                print(f"  {nome}: {valore:.4f}")
        print("\n✅ Processo Completato.")


# --- DEFINIZIONE E ASSEMBLAGGIO INTERFACCIA UTENTE ---
# (Assicurati che tutti i widget individuali siano definiti in una cella precedente)
style_descr_initial = {'description_width': 'initial'}
header_html = widgets.HTML("<h1>Tool di Analisi Finanziaria e Rischio</h1><p>Configura le serie storiche...</p><hr>")
config_generale_box = widgets.VBox([widgets.HTML("<h2>Configurazione Generale</h2>"), tipo_analisi, widgets.HBox([start_date_picker, end_date_picker])], layout=widgets.Layout(margin='0 0 10px 0'))
box_serie1_widgets = widgets.VBox([widgets.HTML("<h3>Serie 1</h3>"), fonte1_dropdown, ticker1_input, gsheet_id_s1_input, gsheet_tab_s1_input, file1_upload], layout=widgets.Layout(padding="0 0 10px 0"))
box_serie2_widgets = widgets.VBox([widgets.HTML("<h3>Serie 2 (per Confronto)</h3>"), fonte2_dropdown, ticker2_input, gsheet_id_s2_input, gsheet_tab_s2_input, file2_upload], layout=widgets.Layout(padding="0 0 10px 0"))
box_data_sources = widgets.HBox([box_serie1_widgets, box_serie2_widgets], layout=widgets.Layout(justify_content='space-around'))
box_garch_settings = widgets.VBox([widgets.HTML("<h2>Impostazioni Modello e Output</h2>"), scelta_distribuzione, scelta_alpha, scelta_giorni_forecast, num_sim_mc_widget], layout=widgets.Layout(margin='10px 0'))
box_export_settings = widgets.VBox([formato_export, nome_file_export], layout=widgets.Layout(margin='10px 0'))
box_model_output_settings = widgets.HBox([box_garch_settings, box_export_settings], layout=widgets.Layout(justify_content='space-between'))

# Creazione del pulsante unico
btn_avvia.description = "Esegui Analisi Completa"
btn_avvia.button_style = 'success'
btn_avvia.icon = 'cogs'
# Rimuoviamo btn_montecarlo dal box
box_buttons = widgets.HBox([btn_avvia], layout=widgets.Layout(justify_content='center', margin='20px 0'))

# Collegamento del pulsante unico alla funzione
btn_avvia.on_click(on_btn_esegui_tutto_clicked)

accordion_outputs = widgets.Accordion(children=[output_principale])
accordion_outputs.set_title(0, 'Risultati e Log Analisi')
accordion_outputs.set_title(1, 'Output Secondario (non più usato)')
accordion_outputs.selected_index = None

# Assemblaggio del layout finale
interfaccia_completa_layout = widgets.VBox([
    header_html, config_generale_box, box_data_sources,
    box_model_output_settings, box_buttons, widgets.HTML("<hr>"),
    accordion_outputs
])

# Visualizzazione Finale
display(interfaccia_completa_layout)
aggiorna_visibilita(None)
print("Interfaccia utente pronta.")

Interfaccia utente pronta.

--- ⏳ Caricamento Dati Serie 1 ---
Caricamento dati da Google Sheet ID: '1ukhNTZC0eJpfFF4V1_S9ZN19jM5xqzjhHx2V3PD8leo', foglio: 'ETF_EIMI'...
--- DEBUG (load_google_sheets): Numero di righe lette da get_all_values(): 2732
--- DEBUG (load_google_sheets): Prima riga letta (header?): ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
--- DEBUG (load_google_sheets): Seconda riga letta (primi dati?): ['24/07/2014 17.26.00', '19,61', '19,62', '19,61', '19,62']
--- DEBUG (load_google_sheets): Prime 5 righe grezze lette ---
                  Date   Open   High    Low  Close Volume
0  24/07/2014 17.26.00  19,61  19,62  19,61  19,62    700
1  28/07/2014 17.26.00  19,77  19,77   19,6   19,6    480
2  29/07/2014 17.26.00  19,76  19,76  19,76  19,76     10
3  30/07/2014 17.26.00  19,72  19,76  19,72  19,76     60
4  31/07/2014 17.26.00  19,38  19,38  19,25  19,25    400
--- DEBUG (load_google_sheets): Tipi di dato iniziali ---
Date      object
Open      object
High      


📊 Metriche di Performance per Serie 1
  Sharpe Ratio: 0.4158
  Sortino Ratio: 0.5781
  Calmar Ratio: 0.3193
  Max Drawdown: -0.1915
  Annualized Return: 0.0612

✅ Processo Completato.

📊 Metriche di Performance per Serie 1
Sharpe Ratio: 0.4158
Sortino Ratio: 0.5781
Calmar Ratio: 0.3193
Max Drawdown: -0.1915
Annualized Return: 0.0612
